step 0

In [1]:
import os
import json
import subprocess
import sys
import platform
import random
from datetime import datetime, timezone
import numpy as np
import torch

os.makedirs('logs', exist_ok=True)

nvidia_smi_output = subprocess.run(['nvidia-smi', '-q'], capture_output=True, text=True)
with open('logs/nvidia_smi.txt', 'w') as f:
    f.write(nvidia_smi_output.stdout)

subprocess.run(['pip', 'freeze'], stdout=open('logs/pip_freeze.txt', 'w'))

import pynvml
pynvml.nvmlInit()

try:
    device_count = pynvml.nvmlDeviceGetCount()
    handle = pynvml.nvmlDeviceGetHandleByIndex(0) if device_count > 0 else None

    _to_str = lambda x: x.decode("utf-8") if isinstance(x, (bytes, bytearray)) else str(x)

    if handle:
        gpu_name = _to_str(pynvml.nvmlDeviceGetName(handle))
        driver_version = _to_str(pynvml.nvmlSystemGetDriverVersion())

        memory_info = pynvml.nvmlDeviceGetMemoryInfo(handle)
        power_limit = pynvml.nvmlDeviceGetPowerManagementLimit(handle) / 1000.0

        try:
            pstate = pynvml.nvmlDeviceGetPerformanceState(handle)
        except:
            pstate = "N/A"

        try:
            _ = pynvml.nvmlDeviceGetTotalEnergyConsumption(handle)
            energy_support = True
            energy_note = "TotalEnergyConsumption supported"
        except pynvml.NVMLError:
            energy_support = False
            energy_note = "TotalEnergyConsumption NOT supported, will use power sampling integration"

        gpu_info = {
            "name": gpu_name,
            "driver_version": driver_version,
            "device_count": device_count,
            "memory_total_mb": memory_info.total // (1024**2),
            "memory_used_mb": memory_info.used // (1024**2),
            "power_limit_w": power_limit,
            "performance_state": str(pstate)
        }
    else:
        gpu_info = {
            "name": "N/A",
            "driver_version": "N/A",
            "device_count": 0
        }
        energy_support = False
        energy_note = "No NVIDIA GPU detected"

finally:
    pynvml.nvmlShutdown()

cuda_available = torch.cuda.is_available()
cuda_version = torch.version.cuda if cuda_available else None
gpu_info["cuda_version"] = cuda_version
gpu_info["cuda_available"] = cuda_available

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if cuda_available:
    torch.cuda.manual_seed_all(42)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True)
    os.environ.setdefault("CUBLAS_WORKSPACE_CONFIG", ":4096:8")

env_info = {
    "timestamp": datetime.now(timezone.utc).isoformat(),
    "python_version": sys.version,
    "platform": platform.platform(),
    "gpu": gpu_info,
    "energy_counter": {
        "supported": energy_support,
        "units": "mJ",
        "note": energy_note
    },
    "library_versions": {
        "torch": torch.__version__,
        "numpy": np.__version__
    },
    "random_seeds": {
        "python": 42,
        "numpy": 42,
        "torch": 42
    },
    "reproducibility": {
        "cudnn_deterministic": cuda_available,
        "cudnn_benchmark": False,
        "deterministic_algorithms": cuda_available
    }
}

with open('logs/env.json', 'w') as f:
    json.dump(env_info, f, indent=2)

print("=" * 70)
print("ENVIRONMENT FINGERPRINT")
print("=" * 70)
print(f"\nTimestamp: {env_info['timestamp']}")
print(f"\nPython:    {sys.version.split()[0]}")
print(f"Platform:  {env_info['platform']}")
print(f"\n{'GPU Configuration':-<70}")
print(f"  Name:          {gpu_info['name']}")
print(f"  Driver:        {gpu_info['driver_version']}")
print(f"  CUDA:          {cuda_version}")
print(f"  CUDA Avail:    {cuda_available}")
print(f"  Device Count:  {gpu_info['device_count']}")
if handle:
    print(f"  Memory:        {gpu_info['memory_used_mb']}/{gpu_info['memory_total_mb']} MB")
    print(f"  Power Limit:   {gpu_info['power_limit_w']:.1f} W")
    print(f"  P-State:       {gpu_info['performance_state']}")
print(f"\n{'Energy Counter':-<70}")
print(f"  Supported:     {energy_support}")
print(f"  Units:         mJ")
print(f"  Note:          {energy_note}")
print(f"\n{'Library Versions':-<70}")
print(f"  PyTorch:       {torch.__version__}")
print(f"  NumPy:         {np.__version__}")
print(f"\n{'Reproducibility':-<70}")
print(f"  Seeds:         42 (Python/NumPy/Torch)")
print(f"  Deterministic: {'Enabled' if cuda_available else 'N/A (no CUDA)'}")
print("\n" + "=" * 70)
print("✓ Environment fingerprint saved to logs/env.json")
print("✓ NVIDIA-SMI output saved to logs/nvidia_smi.txt")
print("✓ Dependencies saved to logs/pip_freeze.txt")
print("=" * 70)

ENVIRONMENT FINGERPRINT

Timestamp: 2025-11-03T09:51:56.031617+00:00

Python:    3.12.12
Platform:  Linux-6.6.105+-x86_64-with-glibc2.35

GPU Configuration-----------------------------------------------------
  Name:          NVIDIA L4
  Driver:        550.54.15
  CUDA:          12.6
  CUDA Avail:    True
  Device Count:  1
  Memory:        341/23034 MB
  Power Limit:   72.0 W
  P-State:       8

Energy Counter--------------------------------------------------------
  Supported:     True
  Units:         mJ
  Note:          TotalEnergyConsumption supported

Library Versions------------------------------------------------------
  PyTorch:       2.8.0+cu126
  NumPy:         2.0.2

Reproducibility-------------------------------------------------------
  Seeds:         42 (Python/NumPy/Torch)
  Deterministic: Enabled

✓ Environment fingerprint saved to logs/env.json
✓ NVIDIA-SMI output saved to logs/nvidia_smi.txt
✓ Dependencies saved to logs/pip_freeze.txt


step 1

In [2]:
import os
import json
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import StandardScaler
import urllib.request
import zipfile

os.makedirs('configs', exist_ok=True)
os.makedirs('data', exist_ok=True)

print("Downloading UCI HAR Dataset...")
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip"
zip_path = "data/uci_har.zip"
urllib.request.urlretrieve(url, zip_path)

print("Extracting dataset...")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("data/")

data_root = "data/UCI HAR Dataset"

def load_inertial_signals(root_path, split='train'):
    signals = ['body_acc_x', 'body_acc_y', 'body_acc_z',
               'body_gyro_x', 'body_gyro_y', 'body_gyro_z',
               'total_acc_x', 'total_acc_y', 'total_acc_z']

    data_list = []
    for signal in signals:
        filepath = f"{root_path}/{split}/Inertial Signals/{signal}_{split}.txt"
        data = np.loadtxt(filepath)
        data_list.append(data)

    X = np.stack(data_list, axis=2)

    y = np.loadtxt(f"{root_path}/{split}/y_{split}.txt").astype(int) - 1
    subject = np.loadtxt(f"{root_path}/{split}/subject_{split}.txt").astype(int)

    return X, y, subject

print("Loading raw inertial signals...")
train_x, train_y, train_subjects = load_inertial_signals(data_root, 'train')
test_x, test_y, test_subjects = load_inertial_signals(data_root, 'test')

activity_labels = {}
with open(f"{data_root}/activity_labels.txt", 'r') as f:
    for line in f:
        idx, label = line.strip().split()
        activity_labels[int(idx) - 1] = label

print(f"Train shape: {train_x.shape}, Test shape: {test_x.shape}")

gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, val_idx = next(gss.split(train_x, train_y, groups=train_subjects))

val_x = train_x[val_idx]
val_y = train_y[val_idx]
val_subjects = train_subjects[val_idx]

train_x = train_x[train_idx]
train_y = train_y[train_idx]
train_subjects = train_subjects[train_idx]

n_samples_train, window_samples, n_channels = train_x.shape

scaler = StandardScaler()
train_x_reshaped = train_x.reshape(-1, n_channels)
scaler.fit(train_x_reshaped)

train_x = scaler.transform(train_x.reshape(-1, n_channels)).reshape(train_x.shape)
val_x = scaler.transform(val_x.reshape(-1, n_channels)).reshape(val_x.shape)
test_x = scaler.transform(test_x.reshape(-1, n_channels)).reshape(test_x.shape)

n_features = n_channels
n_classes = len(activity_labels)
sampling_rate_hz = 50
window_ms = 2560
overlap = 0.5

refresh_rates_hz = [0.1, 0.5, 1.0, 2.0]

total_samples = len(train_x) + len(val_x) + len(test_x)
actual_train_ratio = len(train_x) / total_samples
actual_val_ratio = len(val_x) / total_samples
actual_test_ratio = len(test_x) / total_samples

config = {
    "dataset": "UCI_HAR",
    "data_format": "raw_inertial_signals",
    "n_samples": {
        "train": len(train_x),
        "val": len(val_x),
        "test": len(test_x)
    },
    "n_features": n_features,
    "n_classes": n_classes,
    "class_labels": activity_labels,
    "window": {
        "window_ms": window_ms,
        "window_samples": window_samples,
        "sampling_rate_hz": sampling_rate_hz,
        "overlap": overlap,
        "note": "Fixed window from dataset, cannot be modified"
    },
    "window_grid_applicable": False,
    "refresh_rates_hz": refresh_rates_hz,
    "data_split": {
        "method": "group_shuffle_split_by_subject",
        "train_ratio": round(actual_train_ratio, 3),
        "val_ratio": round(actual_val_ratio, 3),
        "test_ratio": round(actual_test_ratio, 3),
        "random_seed": 42,
        "n_train_subjects": len(np.unique(train_subjects)),
        "n_val_subjects": len(np.unique(val_subjects)),
        "n_test_subjects": len(np.unique(test_subjects))
    },
    "preprocessing": {
        "standardization": "StandardScaler (fit on train, per-channel)",
        "per_inference_mode": True
    },
    "shape": {
        "train": list(train_x.shape),
        "format": "(n_samples, window_samples, n_channels)"
    }
}

with open('configs/windows.json', 'w') as f:
    json.dump(config, f, indent=2)

np.save('data/train_x.npy', train_x)
np.save('data/train_y.npy', train_y)
np.save('data/val_x.npy', val_x)
np.save('data/val_y.npy', val_y)
np.save('data/test_x.npy', test_x)
np.save('data/test_y.npy', test_y)

split_info = pd.DataFrame({
    'split': ['train', 'val', 'test'],
    'n_samples': [len(train_x), len(val_x), len(test_x)],
    'n_subjects': [len(np.unique(train_subjects)),
                   len(np.unique(val_subjects)),
                   len(np.unique(test_subjects))],
    'window_samples': [window_samples, window_samples, window_samples],
    'n_channels': [n_channels, n_channels, n_channels]
})
split_info.to_csv('data/split_info.csv', index=False)

print("=" * 70)
print("DATA & WINDOW CONFIGURATION")
print("=" * 70)
print(f"\nDataset: {config['dataset']} (Raw Inertial Signals)")
print(f"\n{'Data Splits (Subject-Grouped)':-<70}")
print(f"  Train:     {len(train_x):>6} samples ({len(np.unique(train_subjects)):>2} subjects)")
print(f"  Val:       {len(val_x):>6} samples ({len(np.unique(val_subjects)):>2} subjects)")
print(f"  Test:      {len(test_x):>6} samples ({len(np.unique(test_subjects)):>2} subjects)")
print(f"\n{'Actual Ratios':-<70}")
print(f"  Train/Val/Test: {actual_train_ratio:.3f} / {actual_val_ratio:.3f} / {actual_test_ratio:.3f}")
print(f"\n{'Data Properties':-<70}")
print(f"  Shape:     (N, {window_samples}, {n_channels})")
print(f"  Channels:  {n_channels} (body_acc xyz, body_gyro xyz, total_acc xyz)")
print(f"  Classes:   {n_classes}")
print(f"  Sampling:  {sampling_rate_hz} Hz")
print(f"\n{'Fixed Window (Dataset Native)':-<70}")
print(f"  Duration:  {window_ms} ms ({window_ms/1000:.2f} s)")
print(f"  Samples:   {window_samples}")
print(f"  Overlap:   {overlap*100:.0f}%")
print(f"  Note:      Window cannot be modified (pre-segmented dataset)")
print(f"\n{'Refresh Rate Grid (for mJ/s calculation)':-<70}")
print(f"  {refresh_rates_hz} Hz")
print(f"\n{'Class Labels (0-indexed)':-<70}")
for idx, label in activity_labels.items():
    print(f"  {idx}: {label}")
print("\n" + "=" * 70)
print("✓ Configuration saved to configs/windows.json")
print("✓ Preprocessed data saved to data/*.npy")
print("✓ Split info saved to data/split_info.csv")
print("=" * 70)

Extracting dataset...
Loading raw inertial signals...
Train shape: (7352, 128, 9), Test shape: (2947, 128, 9)
DATA & WINDOW CONFIGURATION

Dataset: UCI_HAR (Raw Inertial Signals)

Data Splits (Subject-Grouped)-----------------------------------------
  Train:       5551 samples (16 subjects)
  Val:         1801 samples ( 5 subjects)
  Test:        2947 samples ( 9 subjects)

Actual Ratios---------------------------------------------------------
  Train/Val/Test: 0.539 / 0.175 / 0.286

Data Properties-------------------------------------------------------
  Shape:     (N, 128, 9)
  Channels:  9 (body_acc xyz, body_gyro xyz, total_acc xyz)
  Classes:   6
  Sampling:  50 Hz

Fixed Window (Dataset Native)-----------------------------------------
  Duration:  2560 ms (2.56 s)
  Samples:   128
  Overlap:   50%
  Note:      Window cannot be modified (pre-segmented dataset)

Refresh Rate Grid (for mJ/s calculation)------------------------------
  [0.1, 0.5, 1.0, 2.0] Hz

Class Labels (0-indexe

step 2

In [3]:
import os
import json
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score, f1_score
import pickle
import random

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

os.makedirs('models', exist_ok=True)
os.makedirs('logs', exist_ok=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

try:
    import cupy as cp
    from cuml.neighbors import KNeighborsClassifier as cuKNN
    from cuml.ensemble import RandomForestClassifier as cuRF
    use_gpu_ml = True
    print("✓ cuML available for GPU-accelerated KNN/RF")
except Exception as e:
    use_gpu_ml = False
    raise RuntimeError("cuML not available. Install with: pip install -q cupy-cuda12x cuml-cu12")

print("Loading data...")
train_x = np.load('data/train_x.npy')
train_y = np.load('data/train_y.npy')
val_x = np.load('data/val_x.npy')
val_y = np.load('data/val_y.npy')
test_x = np.load('data/test_x.npy')
test_y = np.load('data/test_y.npy')

with open('configs/windows.json', 'r') as f:
    config = json.load(f)

n_classes = config['n_classes']
n_samples, seq_len, n_channels = train_x.shape

train_x_flat = train_x.reshape(len(train_x), -1).astype(np.float32)
val_x_flat = val_x.reshape(len(val_x), -1).astype(np.float32)
test_x_flat = test_x.reshape(len(test_x), -1).astype(np.float32)

metrics = []

print("\n" + "=" * 70)
print("MODEL TRAINING")
print("=" * 70)

print("\n[1/6] Training KNN (GPU)...")
Xtr_cp = cp.asarray(train_x_flat)
ytr_cp = cp.asarray(train_y.astype(np.int32))
Xva_cp = cp.asarray(val_x_flat)
Xte_cp = cp.asarray(test_x_flat)

knn = cuKNN(n_neighbors=5, algorithm='brute', metric='euclidean')
knn.fit(Xtr_cp, ytr_cp)

val_pred = cp.asnumpy(knn.predict(Xva_cp))
val_acc = accuracy_score(val_y, val_pred)
val_f1 = f1_score(val_y, val_pred, average='macro')

test_pred = cp.asnumpy(knn.predict(Xte_cp))
test_acc = accuracy_score(test_y, test_pred)
test_f1 = f1_score(test_y, test_pred, average='macro')

print(f"  Val  - Accuracy: {val_acc:.4f}, Macro-F1: {val_f1:.4f}")
print(f"  Test - Accuracy: {test_acc:.4f}, Macro-F1: {test_f1:.4f}")

with open('models/knn_cuml.pkl', 'wb') as f:
    pickle.dump(knn, f)
metrics.append({'model': 'KNN_GPU', 'val_accuracy': val_acc, 'val_macro_f1': val_f1,
                'test_accuracy': test_acc, 'test_macro_f1': test_f1})

print("\n[2/6] Training RandomForest (GPU)...")
rf = cuRF(n_estimators=100, max_depth=20, random_state=42)
rf.fit(Xtr_cp, ytr_cp)

val_pred = cp.asnumpy(rf.predict(Xva_cp))
val_acc = accuracy_score(val_y, val_pred)
val_f1 = f1_score(val_y, val_pred, average='macro')

test_pred = cp.asnumpy(rf.predict(Xte_cp))
test_acc = accuracy_score(test_y, test_pred)
test_f1 = f1_score(test_y, test_pred, average='macro')

print(f"  Val  - Accuracy: {val_acc:.4f}, Macro-F1: {val_f1:.4f}")
print(f"  Test - Accuracy: {test_acc:.4f}, Macro-F1: {test_f1:.4f}")

with open('models/rf_cuml.pkl', 'wb') as f:
    pickle.dump(rf, f)
metrics.append({'model': 'RandomForest_GPU', 'val_accuracy': val_acc, 'val_macro_f1': val_f1,
                'test_accuracy': test_acc, 'test_macro_f1': test_f1})

class TransformerModel(nn.Module):
    def __init__(self, input_dim, num_classes, d_model=64, nhead=4, num_layers=2):
        super().__init__()
        self.embedding = nn.Linear(input_dim, d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead,
                                                     dim_feedforward=256, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(d_model, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = self.transformer(x)
        x = x.mean(dim=1)
        return self.fc(x)

print("\n[3/6] Training TST (Transformer)...")
tst_model = TransformerModel(n_channels, n_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(tst_model.parameters(), lr=0.001)

train_dataset = TensorDataset(torch.FloatTensor(train_x), torch.LongTensor(train_y))
val_dataset = TensorDataset(torch.FloatTensor(val_x), torch.LongTensor(val_y))
test_dataset = TensorDataset(torch.FloatTensor(test_x), torch.LongTensor(test_y))

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

for epoch in range(10):
    tst_model.train()
    for batch_x, batch_y in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        optimizer.zero_grad()
        outputs = tst_model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

tst_model.eval()
val_preds, test_preds = [], []
with torch.no_grad():
    for batch_x, _ in val_loader:
        outputs = tst_model(batch_x.to(device))
        val_preds.append(outputs.argmax(dim=1).cpu())
    for batch_x, _ in test_loader:
        outputs = tst_model(batch_x.to(device))
        test_preds.append(outputs.argmax(dim=1).cpu())

val_pred = torch.cat(val_preds).numpy()
test_pred = torch.cat(test_preds).numpy()

val_acc = accuracy_score(val_y, val_pred)
val_f1 = f1_score(val_y, val_pred, average='macro')
test_acc = accuracy_score(test_y, test_pred)
test_f1 = f1_score(test_y, test_pred, average='macro')

print(f"  Val  - Accuracy: {val_acc:.4f}, Macro-F1: {val_f1:.4f}")
print(f"  Test - Accuracy: {test_acc:.4f}, Macro-F1: {test_f1:.4f}")

torch.save(tst_model.state_dict(), 'models/tst.pt')
metrics.append({'model': 'TST', 'val_accuracy': val_acc, 'val_macro_f1': val_f1,
                'test_accuracy': test_acc, 'test_macro_f1': test_f1})

class MiniROCKET(nn.Module):
    def __init__(self, input_channels, num_classes, num_kernels=1000):
        super().__init__()
        self.kernels = nn.Parameter(torch.randn(num_kernels, input_channels, 9))
        self.fc = nn.Linear(num_kernels * 2, num_classes)

    def forward(self, x):
        x = x.transpose(1, 2)
        conv_out = torch.nn.functional.conv1d(x, self.kernels, padding=4)
        ppv = (conv_out > 0).float().mean(dim=2)
        mx = conv_out.amax(dim=2)
        features = torch.cat([ppv, mx], dim=1)
        return self.fc(features)

print("\n[4/6] Training MiniROCKET...")
mini_model = MiniROCKET(n_channels, n_classes).to(device)
optimizer = optim.Adam(mini_model.parameters(), lr=0.001)

for epoch in range(10):
    mini_model.train()
    for batch_x, batch_y in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        optimizer.zero_grad()
        outputs = mini_model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

mini_model.eval()
val_preds, test_preds = [], []
with torch.no_grad():
    for batch_x, _ in val_loader:
        outputs = mini_model(batch_x.to(device))
        val_preds.append(outputs.argmax(dim=1).cpu())
    for batch_x, _ in test_loader:
        outputs = mini_model(batch_x.to(device))
        test_preds.append(outputs.argmax(dim=1).cpu())

val_pred = torch.cat(val_preds).numpy()
test_pred = torch.cat(test_preds).numpy()

val_acc = accuracy_score(val_y, val_pred)
val_f1 = f1_score(val_y, val_pred, average='macro')
test_acc = accuracy_score(test_y, test_pred)
test_f1 = f1_score(test_y, test_pred, average='macro')

print(f"  Val  - Accuracy: {val_acc:.4f}, Macro-F1: {val_f1:.4f}")
print(f"  Test - Accuracy: {test_acc:.4f}, Macro-F1: {test_f1:.4f}")

torch.save(mini_model.state_dict(), 'models/minirocket.pt')
metrics.append({'model': 'MiniROCKET', 'val_accuracy': val_acc, 'val_macro_f1': val_f1,
                'test_accuracy': test_acc, 'test_macro_f1': test_f1})

class MultiROCKET(nn.Module):
    def __init__(self, input_channels, num_classes, num_kernels=2000):
        super().__init__()
        self.num_kernels = num_kernels
        self.kernels = nn.Parameter(torch.randn(num_kernels, input_channels, 9))
        dilations = torch.randint(1, 4, (num_kernels,))
        self.register_buffer('dilations', dilations)
        self.fc = nn.Linear(num_kernels * 2, num_classes)

    def forward(self, x):
        x = x.transpose(1, 2)
        features_list = []
        for d in [1, 2, 3]:
            mask = self.dilations == d
            if mask.sum() > 0:
                kernels_d = self.kernels[mask]
                conv_out = torch.nn.functional.conv1d(x, kernels_d, padding=4*d, dilation=d)
                ppv = (conv_out > 0).float().mean(dim=2)
                mx = conv_out.amax(dim=2)
                features_list.append(torch.cat([ppv, mx], dim=1))
        features = torch.cat(features_list, dim=1)
        return self.fc(features)

print("\n[5/6] Training MultiROCKET...")
multi_model = MultiROCKET(n_channels, n_classes).to(device)
optimizer = optim.Adam(multi_model.parameters(), lr=0.001)

for epoch in range(10):
    multi_model.train()
    for batch_x, batch_y in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        optimizer.zero_grad()
        outputs = multi_model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

multi_model.eval()
val_preds, test_preds = [], []
with torch.no_grad():
    for batch_x, _ in val_loader:
        outputs = multi_model(batch_x.to(device))
        val_preds.append(outputs.argmax(dim=1).cpu())
    for batch_x, _ in test_loader:
        outputs = multi_model(batch_x.to(device))
        test_preds.append(outputs.argmax(dim=1).cpu())

val_pred = torch.cat(val_preds).numpy()
test_pred = torch.cat(test_preds).numpy()

val_acc = accuracy_score(val_y, val_pred)
val_f1 = f1_score(val_y, val_pred, average='macro')
test_acc = accuracy_score(test_y, test_pred)
test_f1 = f1_score(test_y, test_pred, average='macro')

print(f"  Val  - Accuracy: {val_acc:.4f}, Macro-F1: {val_f1:.4f}")
print(f"  Test - Accuracy: {test_acc:.4f}, Macro-F1: {test_f1:.4f}")

torch.save(multi_model.state_dict(), 'models/multirocket.pt')
metrics.append({'model': 'MultiROCKET', 'val_accuracy': val_acc, 'val_macro_f1': val_f1,
                'test_accuracy': test_acc, 'test_macro_f1': test_f1})

class InceptionModule(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=1)
        self.conv3 = nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1)
        self.conv5 = nn.Conv1d(in_channels, out_channels, kernel_size=5, padding=2)
        self.pool = nn.Sequential(nn.MaxPool1d(3, stride=1, padding=1),
                                  nn.Conv1d(in_channels, out_channels, kernel_size=1))
        self.bn = nn.BatchNorm1d(out_channels * 4)

    def forward(self, x):
        x1 = self.conv1(x)
        x2 = self.conv3(x)
        x3 = self.conv5(x)
        x4 = self.pool(x)
        out = torch.cat([x1, x2, x3, x4], dim=1)
        return torch.relu(self.bn(out))

class InceptionTime(nn.Module):
    def __init__(self, input_channels, num_classes):
        super().__init__()
        self.inception1 = InceptionModule(input_channels, 32)
        self.inception2 = InceptionModule(128, 64)
        self.gap = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(256, num_classes)

    def forward(self, x):
        x = x.transpose(1, 2)
        x = self.inception1(x)
        x = self.inception2(x)
        x = self.gap(x).squeeze(-1)
        return self.fc(x)

print("\n[6/6] Training InceptionTime...")
inception_model = InceptionTime(n_channels, n_classes).to(device)
optimizer = optim.Adam(inception_model.parameters(), lr=0.001)

for epoch in range(10):
    inception_model.train()
    for batch_x, batch_y in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        optimizer.zero_grad()
        outputs = inception_model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

inception_model.eval()
val_preds, test_preds = [], []
with torch.no_grad():
    for batch_x, _ in val_loader:
        outputs = inception_model(batch_x.to(device))
        val_preds.append(outputs.argmax(dim=1).cpu())
    for batch_x, _ in test_loader:
        outputs = inception_model(batch_x.to(device))
        test_preds.append(outputs.argmax(dim=1).cpu())

val_pred = torch.cat(val_preds).numpy()
test_pred = torch.cat(test_preds).numpy()

val_acc = accuracy_score(val_y, val_pred)
val_f1 = f1_score(val_y, val_pred, average='macro')
test_acc = accuracy_score(test_y, test_pred)
test_f1 = f1_score(test_y, test_pred, average='macro')

print(f"  Val  - Accuracy: {val_acc:.4f}, Macro-F1: {val_f1:.4f}")
print(f"  Test - Accuracy: {test_acc:.4f}, Macro-F1: {test_f1:.4f}")

torch.save(inception_model.state_dict(), 'models/inceptiontime.pt')
metrics.append({'model': 'InceptionTime', 'val_accuracy': val_acc, 'val_macro_f1': val_f1,
                'test_accuracy': test_acc, 'test_macro_f1': test_f1})

metrics_df = pd.DataFrame(metrics)
metrics_df.to_csv('logs/train_metrics.csv', index=False)

print("\n" + "=" * 70)
print("TRAINING SUMMARY")
print("=" * 70)
print(f"{'Model':<20} {'Val Acc':<10} {'Val F1':<10} {'Test Acc':<10} {'Test F1':<10}")
print("-" * 70)
for _, row in metrics_df.iterrows():
    print(f"{row['model']:<20} {row['val_accuracy']:.4f}     {row['val_macro_f1']:.4f}     "
          f"{row['test_accuracy']:.4f}      {row['test_macro_f1']:.4f}")
print("\n" + "=" * 70)
print("✓ Models saved to models/")
print("✓ Training metrics saved to logs/train_metrics.csv")
print("✓ Random seed: 42 (fixed)")
print("=" * 70)

Using device: cuda
✓ cuML available for GPU-accelerated KNN/RF
Loading data...

MODEL TRAINING

[1/6] Training KNN (GPU)...
  Val  - Accuracy: 0.7518, Macro-F1: 0.7347
  Test - Accuracy: 0.6698, Macro-F1: 0.6555

[2/6] Training RandomForest (GPU)...
  Val  - Accuracy: 0.9445, Macro-F1: 0.9445
  Test - Accuracy: 0.8483, Macro-F1: 0.8461

[3/6] Training TST (Transformer)...
  Val  - Accuracy: 0.9378, Macro-F1: 0.9314
  Test - Accuracy: 0.8799, Macro-F1: 0.8748

[4/6] Training MiniROCKET...
  Val  - Accuracy: 0.9733, Macro-F1: 0.9740
  Test - Accuracy: 0.9287, Macro-F1: 0.9300

[5/6] Training MultiROCKET...
  Val  - Accuracy: 0.9761, Macro-F1: 0.9766
  Test - Accuracy: 0.9406, Macro-F1: 0.9411

[6/6] Training InceptionTime...
  Val  - Accuracy: 0.9783, Macro-F1: 0.9788
  Test - Accuracy: 0.9369, Macro-F1: 0.9371

TRAINING SUMMARY
Model                Val Acc    Val F1     Test Acc   Test F1   
----------------------------------------------------------------------
KNN_GPU              0.75

step 3

In [4]:
import os
import numpy as np
import torch
import torch.nn as nn
import pickle
import cupy as cp
from abc import ABC, abstractmethod

USE_GPU_ONLY = True
if USE_GPU_ONLY:
    if not torch.cuda.is_available():
        raise RuntimeError("Step 3 要求 GPU 推理，但当前未检测到 CUDA。")
    if cp.cuda.runtime.getDeviceCount() < 1:
        raise RuntimeError("未检测到可用的 CUDA 设备（CuPy）。")
    cp.cuda.Device(0).use()
    torch.cuda.set_device(0)
device = torch.device('cuda')

class InferenceAdapter(ABC):
    def __init__(self, model_name):
        self.model_name = model_name
        self.model = None
        self.warmed_up = False
        self.expected_shape = None

    def _validate(self, X, expect_channels=None, dtype=np.float32):
        X = np.asarray(X)
        if X.ndim != 3:
            raise ValueError(f"期望 X 形状 [B,T,C]，实际 {X.shape}")
        if self.expected_shape is not None and tuple(X.shape) != self.expected_shape:
            raise ValueError(f"输入形状漂移：期望 {self.expected_shape}，实际 {X.shape}")
        if expect_channels is not None and X.shape[2] != expect_channels:
            raise ValueError(f"C 维不一致：期望 {expect_channels}，实际 {X.shape[2]}")
        if X.dtype != dtype:
            X = X.astype(dtype, copy=False)
        return X

    @abstractmethod
    def load_model(self):
        pass

    @abstractmethod
    def infer_one_batch(self, X):
        pass

    def warmup(self, input_shape, n_warmup=20):
        self.expected_shape = tuple(input_shape)
        dummy_input = np.random.randn(*self.expected_shape).astype(np.float32)
        for _ in range(n_warmup):
            _ = self.infer_one_batch(dummy_input)
        self.warmed_up = True

    def infer_repeated(self, X, n_repeat=100):
        results = []
        for _ in range(n_repeat):
            result = self.infer_one_batch(X)
            results.append(result)
        return results


class KNNAdapter(InferenceAdapter):
    def __init__(self, n_channels=9):
        super().__init__("KNN_GPU")
        self.n_channels = n_channels
        self.expected_flat = None
        self.load_model()

    def load_model(self):
        with open('models/knn_cuml.pkl', 'rb') as f:
            self.model = pickle.load(f)
        self.expected_flat = getattr(self.model, "n_features_in_", None) or getattr(self.model, "n_cols", None)

    def infer_one_batch(self, X):
        X = self._validate(X, self.n_channels)
        B, T, C = X.shape
        X_flat = X.reshape(B, -1)
        if self.expected_flat is not None and X_flat.shape[1] != self.expected_flat:
            raise ValueError(f"KNN 输入维度与训练不一致：期望 {self.expected_flat}，实际 {X_flat.shape[1]}")
        with cp.cuda.Device(0):
            X_cp = cp.asarray(X_flat)
            probs_cp = self.model.predict_proba(X_cp)
        cp.cuda.runtime.deviceSynchronize()
        probs = cp.asnumpy(probs_cp)
        return probs.astype(np.float32, copy=False)


class RandomForestAdapter(InferenceAdapter):
    def __init__(self, n_channels=9):
        super().__init__("RandomForest_GPU")
        self.n_channels = n_channels
        self.expected_flat = None
        self.load_model()

    def load_model(self):
        with open('models/rf_cuml.pkl', 'rb') as f:
            self.model = pickle.load(f)
        self.expected_flat = getattr(self.model, "n_features_in_", None) or getattr(self.model, "n_cols", None)

    def infer_one_batch(self, X):
        X = self._validate(X, self.n_channels)
        B, T, C = X.shape
        X_flat = X.reshape(B, -1)
        if self.expected_flat is not None and X_flat.shape[1] != self.expected_flat:
            raise ValueError(f"RandomForest 输入维度与训练不一致：期望 {self.expected_flat}，实际 {X_flat.shape[1]}")
        with cp.cuda.Device(0):
            X_cp = cp.asarray(X_flat)
            probs_cp = self.model.predict_proba(X_cp)
        cp.cuda.runtime.deviceSynchronize()
        probs = cp.asnumpy(probs_cp)
        return probs.astype(np.float32, copy=False)


class TransformerModel(nn.Module):
    def __init__(self, input_dim, num_classes, d_model=64, nhead=4, num_layers=2):
        super().__init__()
        self.embedding = nn.Linear(input_dim, d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead,
                                                     dim_feedforward=256, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(d_model, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = self.transformer(x)
        x = x.mean(dim=1)
        return self.fc(x)


class TSTAdapter(InferenceAdapter):
    def __init__(self, n_channels=9, n_classes=6):
        super().__init__("TST")
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.load_model()

    def load_model(self):
        self.model = TransformerModel(self.n_channels, self.n_classes).to(device)
        self.model.load_state_dict(torch.load('models/tst.pt', map_location=device))
        self.model.eval()

    def infer_one_batch(self, X):
        X = self._validate(X, self.n_channels)
        X_tensor = torch.FloatTensor(X).to(device)
        with torch.inference_mode():
            logits = self.model(X_tensor)
            probs = torch.softmax(logits, dim=1)
        torch.cuda.synchronize()
        return probs.detach().cpu().numpy().astype(np.float32, copy=False)


class MiniROCKET(nn.Module):
    def __init__(self, input_channels, num_classes, num_kernels=1000):
        super().__init__()
        self.kernels = nn.Parameter(torch.randn(num_kernels, input_channels, 9))
        self.fc = nn.Linear(num_kernels * 2, num_classes)

    def forward(self, x):
        x = x.transpose(1, 2).contiguous()
        conv_out = torch.nn.functional.conv1d(x, self.kernels, padding=4)
        ppv = (conv_out > 0).float().mean(dim=2)
        mx = conv_out.amax(dim=2)
        features = torch.cat([ppv, mx], dim=1)
        return self.fc(features)


class MiniROCKETAdapter(InferenceAdapter):
    def __init__(self, n_channels=9, n_classes=6):
        super().__init__("MiniROCKET")
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.load_model()

    def load_model(self):
        self.model = MiniROCKET(self.n_channels, self.n_classes).to(device)
        self.model.load_state_dict(torch.load('models/minirocket.pt', map_location=device))
        self.model.eval()

    def infer_one_batch(self, X):
        X = self._validate(X, self.n_channels)
        X_tensor = torch.FloatTensor(X).to(device)
        with torch.inference_mode():
            logits = self.model(X_tensor)
            probs = torch.softmax(logits, dim=1)
        torch.cuda.synchronize()
        return probs.detach().cpu().numpy().astype(np.float32, copy=False)


class MultiROCKET(nn.Module):
    def __init__(self, input_channels, num_classes, num_kernels=2000):
        super().__init__()
        self.num_kernels = num_kernels
        self.kernels = nn.Parameter(torch.randn(num_kernels, input_channels, 9))
        dilations = torch.randint(1, 4, (num_kernels,))
        self.register_buffer('dilations', dilations)
        self.fc = nn.Linear(num_kernels * 2, num_classes)

    def forward(self, x):
        x = x.transpose(1, 2).contiguous()
        features_list = []
        for d in [1, 2, 3]:
            mask = self.dilations == d
            if mask.any():
                kernels_d = self.kernels[mask]
                conv_out = torch.nn.functional.conv1d(x, kernels_d, padding=4*d, dilation=d)
                ppv = (conv_out > 0).float().mean(dim=2)
                mx = conv_out.amax(dim=2)
                features_list.append(torch.cat([ppv, mx], dim=1))
        features = torch.cat(features_list, dim=1)
        return self.fc(features)


class MultiROCKETAdapter(InferenceAdapter):
    def __init__(self, n_channels=9, n_classes=6):
        super().__init__("MultiROCKET")
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.load_model()

    def load_model(self):
        self.model = MultiROCKET(self.n_channels, self.n_classes).to(device)
        self.model.load_state_dict(torch.load('models/multirocket.pt', map_location=device))
        self.model.eval()

    def infer_one_batch(self, X):
        X = self._validate(X, self.n_channels)
        X_tensor = torch.FloatTensor(X).to(device)
        with torch.inference_mode():
            logits = self.model(X_tensor)
            probs = torch.softmax(logits, dim=1)
        torch.cuda.synchronize()
        return probs.detach().cpu().numpy().astype(np.float32, copy=False)


class InceptionModule(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=1)
        self.conv3 = nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1)
        self.conv5 = nn.Conv1d(in_channels, out_channels, kernel_size=5, padding=2)
        self.pool = nn.Sequential(nn.MaxPool1d(3, stride=1, padding=1),
                                  nn.Conv1d(in_channels, out_channels, kernel_size=1))
        self.bn = nn.BatchNorm1d(out_channels * 4)

    def forward(self, x):
        x1 = self.conv1(x)
        x2 = self.conv3(x)
        x3 = self.conv5(x)
        x4 = self.pool(x)
        out = torch.cat([x1, x2, x3, x4], dim=1)
        return torch.relu(self.bn(out))


class InceptionTime(nn.Module):
    def __init__(self, input_channels, num_classes):
        super().__init__()
        self.inception1 = InceptionModule(input_channels, 32)
        self.inception2 = InceptionModule(128, 64)
        self.gap = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(256, num_classes)

    def forward(self, x):
        x = x.transpose(1, 2).contiguous()
        x = self.inception1(x)
        x = self.inception2(x)
        x = self.gap(x).squeeze(-1)
        return self.fc(x)


class InceptionTimeAdapter(InferenceAdapter):
    def __init__(self, n_channels=9, n_classes=6):
        super().__init__("InceptionTime")
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.load_model()

    def load_model(self):
        self.model = InceptionTime(self.n_channels, self.n_classes).to(device)
        self.model.load_state_dict(torch.load('models/inceptiontime.pt', map_location=device))
        self.model.eval()

    def infer_one_batch(self, X):
        X = self._validate(X, self.n_channels)
        X_tensor = torch.FloatTensor(X).to(device)
        with torch.inference_mode():
            logits = self.model(X_tensor)
            probs = torch.softmax(logits, dim=1)
        torch.cuda.synchronize()
        return probs.detach().cpu().numpy().astype(np.float32, copy=False)


def create_adapters():
    adapters = {
        'KNN': KNNAdapter(),
        'RandomForest': RandomForestAdapter(),
        'TST': TSTAdapter(),
        'MiniROCKET': MiniROCKETAdapter(),
        'MultiROCKET': MultiROCKETAdapter(),
        'InceptionTime': InceptionTimeAdapter()
    }
    return adapters


if __name__ == "__main__":
    test_x = np.load('data/test_x.npy')
    batch_size = 32
    test_batch = test_x[:batch_size]

    adapters = create_adapters()

    print("=" * 70)
    print("INFERENCE ADAPTER TEST")
    print("=" * 70)
    print(f"\nInput shape: {test_batch.shape}")
    print(f"Device: {device}")

    for name, adapter in adapters.items():
        print(f"\n[{name}]")

        print("  Warming up...")
        adapter.warmup(test_batch.shape)

        print("  Running inference...")
        probs = adapter.infer_one_batch(test_batch)
        preds = np.argmax(probs, axis=1)

        print(f"  Output shape: {probs.shape}")
        print(f"  Predictions: {preds[:10]}")
        print(f"  Prob range: [{probs.min():.4f}, {probs.max():.4f}]")

    print("\n" + "=" * 70)
    print("✓ All adapters tested successfully")
    print("=" * 70)

INFERENCE ADAPTER TEST

Input shape: (32, 128, 9)
Device: cuda

[KNN]
  Warming up...
  Running inference...
  Output shape: (32, 6)
  Predictions: [4 4 4 4 4 4 4 4 4 4]
  Prob range: [0.0000, 1.0000]

[RandomForest]
  Warming up...
  Running inference...
  Output shape: (32, 6)
  Predictions: [4 4 4 4 4 4 4 4 4 4]
  Prob range: [0.0000, 1.0000]

[TST]
  Warming up...
  Running inference...
  Output shape: (32, 6)
  Predictions: [4 4 4 4 4 4 4 4 4 4]
  Prob range: [0.0000, 0.9985]

[MiniROCKET]
  Warming up...
  Running inference...
  Output shape: (32, 6)
  Predictions: [4 4 4 4 4 4 4 4 4 4]
  Prob range: [0.0000, 0.9987]

[MultiROCKET]
  Warming up...
  Running inference...
  Output shape: (32, 6)
  Predictions: [4 4 4 4 4 4 4 4 4 4]
  Prob range: [0.0000, 1.0000]

[InceptionTime]
  Warming up...
  Running inference...
  Output shape: (32, 6)
  Predictions: [4 4 4 4 4 4 4 4 4 4]
  Prob range: [0.0000, 0.9946]

✓ All adapters tested successfully


step 4

In [5]:
import os
import json
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import pickle
from collections import defaultdict

os.makedirs('logs', exist_ok=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

macs_counter = {'total': 0}
_outproj_called = defaultdict(bool)

class TransformerModel(nn.Module):
    def __init__(self, input_dim, num_classes, d_model=64, nhead=4, num_layers=2):
        super().__init__()
        self.embedding = nn.Linear(input_dim, d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead,
                                                     dim_feedforward=256, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(d_model, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = self.transformer(x)
        x = x.mean(dim=1)
        return self.fc(x)

class MiniROCKET(nn.Module):
    def __init__(self, input_channels, num_classes, num_kernels=1000):
        super().__init__()
        self.num_kernels = num_kernels
        self.input_channels = input_channels
        self.kernels = nn.Parameter(torch.randn(num_kernels, input_channels, 9))
        self.fc = nn.Linear(num_kernels * 2, num_classes)

    def forward(self, x):
        x = x.transpose(1, 2).contiguous()
        conv_out = torch.nn.functional.conv1d(x, self.kernels, padding=4)
        ppv = (conv_out > 0).float().mean(dim=2)
        mx = conv_out.amax(dim=2)
        features = torch.cat([ppv, mx], dim=1)
        return self.fc(features)

class MultiROCKET(nn.Module):
    def __init__(self, input_channels, num_classes, num_kernels=2000):
        super().__init__()
        self.num_kernels = num_kernels
        self.input_channels = input_channels
        self.kernels = nn.Parameter(torch.randn(num_kernels, input_channels, 9))
        dilations = torch.randint(1, 4, (num_kernels,))
        self.register_buffer('dilations', dilations)
        self.fc = nn.Linear(num_kernels * 2, num_classes)

    def forward(self, x):
        x = x.transpose(1, 2).contiguous()
        features_list = []
        for d in [1, 2, 3]:
            mask = self.dilations == d
            if mask.any():
                kernels_d = self.kernels[mask]
                conv_out = torch.nn.functional.conv1d(x, kernels_d, padding=4*d, dilation=d)
                ppv = (conv_out > 0).float().mean(dim=2)
                mx = conv_out.amax(dim=2)
                features_list.append(torch.cat([ppv, mx], dim=1))
        features = torch.cat(features_list, dim=1)
        return self.fc(features)

class InceptionModule(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=1)
        self.conv3 = nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1)
        self.conv5 = nn.Conv1d(in_channels, out_channels, kernel_size=5, padding=2)
        self.pool = nn.Sequential(nn.MaxPool1d(3, stride=1, padding=1),
                                  nn.Conv1d(in_channels, out_channels, kernel_size=1))
        self.bn = nn.BatchNorm1d(out_channels * 4)

    def forward(self, x):
        x1 = self.conv1(x)
        x2 = self.conv3(x)
        x3 = self.conv5(x)
        x4 = self.pool(x)
        out = torch.cat([x1, x2, x3, x4], dim=1)
        return torch.relu(self.bn(out))

class InceptionTime(nn.Module):
    def __init__(self, input_channels, num_classes):
        super().__init__()
        self.inception1 = InceptionModule(input_channels, 32)
        self.inception2 = InceptionModule(128, 64)
        self.gap = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(256, num_classes)

    def forward(self, x):
        x = x.transpose(1, 2).contiguous()
        x = self.inception1(x)
        x = self.inception2(x)
        x = self.gap(x).squeeze(-1)
        return self.fc(x)

test_x = np.load('data/test_x.npy')
batch_size = 32
test_batch = test_x[:batch_size]

B, seq_len, n_channels = test_batch.shape
n_classes = 6

def make_mark_out_proj(mha_id):
    def _mark_out_proj(m, x, y):
        _outproj_called[mha_id] = True
    return _mark_out_proj

def count_conv1d(m, x, y):
    batch = y.size(0)
    kernel_size = m.kernel_size[0]
    in_ch = m.in_channels
    out_ch = m.out_channels
    out_size = y.size(2)
    groups = m.groups
    macs = batch * out_size * out_ch * (in_ch // groups) * kernel_size
    macs_counter['total'] += macs

def count_linear(m, x, y):
    B = y.size(0)
    out_feat = m.out_features
    in_feat = m.in_features
    num_pos = y.numel() // (B * out_feat)
    macs = B * num_pos * in_feat * out_feat
    macs_counter['total'] += macs

def count_batchnorm(m, x, y):
    batch = y.size(0)
    elements = y.numel() / batch
    macs = 2 * batch * elements
    macs_counter['total'] += macs

def count_multihead_attention(m, x, y):
    if isinstance(m, nn.MultiheadAttention):
        B, N, d = x[0].size()
        H = m.num_heads
        d_k = d // H
        macs = 3 * B * N * d * d
        macs += 2 * B * H * N * N * d_k
        if not _outproj_called[id(m)]:
            macs += B * N * d * d
        macs_counter['total'] += macs

def count_layernorm(m, x, y):
    batch = y.size(0)
    elements = y.numel() / batch
    macs = 5 * batch * elements
    macs_counter['total'] += macs

def count_minirocket(m, x, y):
    if isinstance(m, MiniROCKET):
        B, T, C = x[0].shape
        ks = m.kernels.shape[2]
        L_out = T
        macs = B * L_out * m.num_kernels * C * ks
        macs_counter['total'] += macs

def count_multirocket(m, x, y):
    if isinstance(m, MultiROCKET):
        B, T, C = x[0].shape
        ks = m.kernels.shape[2]
        for d in [1, 2, 3]:
            K_d = int((m.dilations == d).sum().item())
            if K_d > 0:
                L_out = T
                macs = B * L_out * K_d * C * ks
                macs_counter['total'] += macs

def register_hooks(model):
    hooks = []
    for m in model.modules():
        if isinstance(m, nn.MultiheadAttention):
            hooks.append(m.out_proj.register_forward_hook(make_mark_out_proj(id(m))))
            hooks.append(m.register_forward_hook(count_multihead_attention))
        elif isinstance(m, nn.Conv1d):
            hooks.append(m.register_forward_hook(count_conv1d))
        elif isinstance(m, nn.Linear):
            hooks.append(m.register_forward_hook(count_linear))
        elif isinstance(m, (nn.BatchNorm1d, nn.BatchNorm2d)):
            hooks.append(m.register_forward_hook(count_batchnorm))
        elif isinstance(m, nn.LayerNorm):
            hooks.append(m.register_forward_hook(count_layernorm))
        elif isinstance(m, MiniROCKET):
            hooks.append(m.register_forward_hook(count_minirocket))
        elif isinstance(m, MultiROCKET):
            hooks.append(m.register_forward_hook(count_multirocket))
    return hooks

models = {
    'TST': TransformerModel(n_channels, n_classes).to(device),
    'MiniROCKET': MiniROCKET(n_channels, n_classes).to(device),
    'MultiROCKET': MultiROCKET(n_channels, n_classes).to(device),
    'InceptionTime': InceptionTime(n_channels, n_classes).to(device)
}

for name in models:
    models[name].load_state_dict(torch.load(f'models/{name.lower()}.pt', map_location=device))
    models[name].eval()

results = []

print("=" * 70)
print("MACs CALCULATION")
print("=" * 70)

for model_name, model in models.items():
    print(f"\n[{model_name}]")

    _outproj_called.clear()
    macs_counter['total'] = 0
    hooks = register_hooks(model)

    X_tensor = torch.FloatTensor(test_batch).to(device)
    with torch.no_grad():
        _ = model(X_tensor)

    for h in hooks:
        h.remove()

    total_macs = macs_counter['total']
    macs_per_inf = total_macs / batch_size

    print(f"  Total MACs (B={batch_size}): {total_macs:,.0f}")
    print(f"  MACs per inference: {macs_per_inf:,.0f}")

    results.append({
        'model': model_name,
        'batch_size': batch_size,
        'total_macs': total_macs,
        'macs_per_inf': macs_per_inf,
        'method': 'forward_hook',
        'note': ''
    })

print(f"\n[KNN]")
d = seq_len * n_channels
n_train = len(np.load('data/train_x.npy'))

macs_per_inf = 2 * d * n_train
total_macs = macs_per_inf * batch_size

print(f"  d={d}, N_train={n_train}")
print(f"  MACs per inference: {macs_per_inf:,.0f}")
print(f"  Formula: 2 * d * N_train (L2 distance computation)")

results.append({
    'model': 'KNN',
    'batch_size': batch_size,
    'total_macs': total_macs,
    'macs_per_inf': macs_per_inf,
    'method': 'analytical',
    'note': 'L2 distance brute-force'
})

print(f"\n[RandomForest]")

with open('models/rf_cuml.pkl', 'rb') as f:
    rf_model = pickle.load(f)

try:
    n_trees = getattr(rf_model, "n_estimators", getattr(rf_model, "n_estimators_", 100))
except:
    n_trees = 100

avg_depth = 15

print(f"  n_trees={n_trees}, avg_depth≈{avg_depth} (conservative estimate)")

ops_per_inf = n_trees * avg_depth * 2
total_ops = ops_per_inf * batch_size

print(f"  Ops per inference: {ops_per_inf:,.0f}")
print(f"  Formula: n_trees * avg_depth * 2 (comparison converted to MAC-equiv)")

results.append({
    'model': 'RandomForest',
    'batch_size': batch_size,
    'total_macs': total_ops,
    'macs_per_inf': ops_per_inf,
    'method': 'analytical',
    'note': 'Threshold comparisons converted to MAC-equiv (×2)'
})

df = pd.DataFrame(results)
df.to_csv('logs/macs_per_inf.csv', index=False)

print("\n" + "=" * 70)
print("MACs SUMMARY")
print("=" * 70)
print(f"{'Model':<20} {'MACs/inf':<20} {'Method':<15}")
print("-" * 70)
for _, row in df.iterrows():
    print(f"{row['model']:<20} {row['macs_per_inf']:>19,.0f} {row['method']:<15}")

print("\n" + "=" * 70)
print("CALCULATION METHODOLOGY")
print("=" * 70)
print(f"""
UNIT: MACs (Multiply-Accumulate operations)
  - 1 MAC = 1 multiply + 1 add
  - To convert to FLOPs: multiply by 2

Deep Learning Models (forward_hook):
  Conv1d:            batch * out_size * out_ch * (in_ch/groups) * kernel_size
  Linear:            B * num_pos * in_feat * out_feat
                     where num_pos = y.numel() // (B * out_feat)
                     handles both (B, N, feat) and (B, feat) automatically
  BatchNorm:         2 * batch * elements (mean + variance normalization)
  LayerNorm:         5 * batch * elements (mean, variance, scale, shift)
  MultiheadAttention:
    - QKV projection: 3 * B * N * d * d (functional linear, not via Linear hook)
    - Attention:      2 * B * H * N * N * d_k (QK^T + Attn@V)
    - out_proj:       B * N * d * d (added if not counted by Linear hook)
    - Note: FFN counted separately by Linear hook
  MiniROCKET:        B * T * num_kernels * in_ch * 9
  MultiROCKET:       Sum over dilations [1,2,3] of B * T * K_d * in_ch * 9

KNN (analytical approximation):
  2 * d * N_train
  Assumption: L2 distance = subtract + square + sum (≈2d MACs per pair)
  Brute-force search over all training samples
  d={d}, N_train={n_train}
  Note: Top-k maintenance overhead (N*log k comparisons) is negligible

RandomForest (analytical approximation):
  n_trees * avg_depth * 2
  Assumption: Each node performs 1 threshold comparison
  Converted to MAC-equiv by multiplying by 2 for cross-model comparison
  avg_depth ≈ {avg_depth} (conservative estimate)
  n_trees={n_trees}
  Note: True comparison count = n_trees * avg_depth

Input shape: {test_batch.shape} - [batch, seq_len, channels]
Batch size for measurement: {batch_size}
All MACs normalized to per-inference basis.

Note: MaxPool, AdaptiveAvgPool, ReLU overheads not counted (negligible).
""")

print("=" * 70)
print("✓ MACs statistics saved to logs/macs_per_inf.csv")
print("✓ Column 'macs_per_inf' represents MACs (not FLOPs)")
print("=" * 70)

MACs CALCULATION

[TST]
  Total MACs (B=32): 544,485,376
  MACs per inference: 17,015,168

[MiniROCKET]
  Total MACs (B=32): 332,160,000
  MACs per inference: 10,380,000

[MultiROCKET]
  Total MACs (B=32): 664,320,000
  MACs per inference: 20,760,000

[InceptionTime]
  Total MACs (B=32): 350,535,680
  MACs per inference: 10,954,240

[KNN]
  d=1152, N_train=5551
  MACs per inference: 12,789,504
  Formula: 2 * d * N_train (L2 distance computation)

[RandomForest]
  n_trees=100, avg_depth≈15 (conservative estimate)
  Ops per inference: 3,000
  Formula: n_trees * avg_depth * 2 (comparison converted to MAC-equiv)

MACs SUMMARY
Model                MACs/inf             Method         
----------------------------------------------------------------------
TST                           17,015,168 forward_hook   
MiniROCKET                    10,380,000 forward_hook   
MultiROCKET                   20,760,000 forward_hook   
InceptionTime                 10,954,240 forward_hook   
KNN          

step 5

In [6]:
import os
import json
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import pickle

os.makedirs('logs', exist_ok=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class TransformerModel(nn.Module):
    def __init__(self, input_dim, num_classes, d_model=64, nhead=4, num_layers=2):
        super().__init__()
        self.embedding = nn.Linear(input_dim, d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead,
                                                     dim_feedforward=256, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(d_model, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = self.transformer(x)
        x = x.mean(dim=1)
        return self.fc(x)

class MiniROCKET(nn.Module):
    def __init__(self, input_channels, num_classes, num_kernels=1000):
        super().__init__()
        self.num_kernels = num_kernels
        self.input_channels = input_channels
        self.kernels = nn.Parameter(torch.randn(num_kernels, input_channels, 9))
        self.fc = nn.Linear(num_kernels * 2, num_classes)

    def forward(self, x):
        x = x.transpose(1, 2).contiguous()
        conv_out = torch.nn.functional.conv1d(x, self.kernels, padding=4)
        ppv = (conv_out > 0).float().mean(dim=2)
        mx = conv_out.amax(dim=2)
        features = torch.cat([ppv, mx], dim=1)
        return self.fc(features)

class MultiROCKET(nn.Module):
    def __init__(self, input_channels, num_classes, num_kernels=2000):
        super().__init__()
        self.num_kernels = num_kernels
        self.input_channels = input_channels
        self.kernels = nn.Parameter(torch.randn(num_kernels, input_channels, 9))
        dilations = torch.randint(1, 4, (num_kernels,))
        self.register_buffer('dilations', dilations)
        self.fc = nn.Linear(num_kernels * 2, num_classes)

    def forward(self, x):
        x = x.transpose(1, 2).contiguous()
        features_list = []
        for d in [1, 2, 3]:
            mask = self.dilations == d
            if mask.any():
                kernels_d = self.kernels[mask]
                conv_out = torch.nn.functional.conv1d(x, kernels_d, padding=4*d, dilation=d)
                ppv = (conv_out > 0).float().mean(dim=2)
                mx = conv_out.amax(dim=2)
                features_list.append(torch.cat([ppv, mx], dim=1))
        features = torch.cat(features_list, dim=1)
        return self.fc(features)

class InceptionModule(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=1)
        self.conv3 = nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1)
        self.conv5 = nn.Conv1d(in_channels, out_channels, kernel_size=5, padding=2)
        self.pool = nn.Sequential(nn.MaxPool1d(3, stride=1, padding=1),
                                  nn.Conv1d(in_channels, out_channels, kernel_size=1))
        self.bn = nn.BatchNorm1d(out_channels * 4)

    def forward(self, x):
        x1 = self.conv1(x)
        x2 = self.conv3(x)
        x3 = self.conv5(x)
        x4 = self.pool(x)
        out = torch.cat([x1, x2, x3, x4], dim=1)
        return torch.relu(self.bn(out))

class InceptionTime(nn.Module):
    def __init__(self, input_channels, num_classes):
        super().__init__()
        self.inception1 = InceptionModule(input_channels, 32)
        self.inception2 = InceptionModule(128, 64)
        self.gap = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(256, num_classes)

    def forward(self, x):
        x = x.transpose(1, 2).contiguous()
        x = self.inception1(x)
        x = self.inception2(x)
        x = self.gap(x).squeeze(-1)
        return self.fc(x)

def count_model_bytes(model, input_shape, add_initial_transpose_write=False):
    bytes_per_elem = 4
    B, T, C = input_shape

    input_read = B * T * C * bytes_per_elem

    param_read = 0
    for p in model.parameters():
        param_read += p.numel() * bytes_per_elem

    activation_read = 0
    activation_write = 0

    with torch.no_grad():
        x = torch.randn(input_shape).to(device)

        def hook_fn(m, inp, out):
            nonlocal activation_read, activation_write
            if isinstance(inp, tuple):
                inp = inp[0]
            activation_read += inp.numel() * bytes_per_elem
            activation_write += out.numel() * bytes_per_elem

        hooks = []
        for m in model.modules():
            if len(list(m.children())) == 0 and not isinstance(m, (nn.MultiheadAttention,)):
                hooks.append(m.register_forward_hook(hook_fn))

        output = model(x)

        for h in hooks:
            h.remove()

    output_write = output.numel() * bytes_per_elem

    activation_write -= output_write

    if add_initial_transpose_write:
        activation_write += B * C * T * bytes_per_elem

    return {
        'input_read': input_read,
        'param_read': param_read,
        'activation_read': activation_read,
        'activation_write': activation_write,
        'output_write': output_write
    }

def count_minirocket_bytes(model, input_shape):
    bytes_per_elem = 4
    B, T, C = input_shape

    input_read = B * T * C * bytes_per_elem

    kernels_read = model.num_kernels * C * 9 * bytes_per_elem
    fc_weight_read = model.fc.weight.numel() * bytes_per_elem
    fc_bias_read = model.fc.bias.numel() * bytes_per_elem if model.fc.bias is not None else 0
    param_read = kernels_read + fc_weight_read + fc_bias_read

    transposed_write = B * C * T * bytes_per_elem
    conv_in_read = B * C * T * bytes_per_elem
    conv_out_write = B * model.num_kernels * T * bytes_per_elem
    conv_out_read_ppv = B * model.num_kernels * T * bytes_per_elem
    conv_out_read_mx = B * model.num_kernels * T * bytes_per_elem
    features_write = B * (model.num_kernels * 2) * bytes_per_elem
    features_read = B * (model.num_kernels * 2) * bytes_per_elem

    activation_read = conv_in_read + conv_out_read_ppv + conv_out_read_mx + features_read
    activation_write = transposed_write + conv_out_write + features_write

    output_write = B * model.fc.out_features * bytes_per_elem

    return {
        'input_read': input_read,
        'param_read': param_read,
        'activation_read': activation_read,
        'activation_write': activation_write,
        'output_write': output_write
    }

def count_multirocket_bytes(model, input_shape):
    bytes_per_elem = 4
    B, T, C = input_shape

    input_read = B * T * C * bytes_per_elem

    kernels_read = model.num_kernels * C * 9 * bytes_per_elem
    fc_weight_read = model.fc.weight.numel() * bytes_per_elem
    fc_bias_read = model.fc.bias.numel() * bytes_per_elem if model.fc.bias is not None else 0
    param_read = kernels_read + fc_weight_read + fc_bias_read

    transposed_write = B * C * T * bytes_per_elem

    conv_in_read = 0
    total_conv_out_write = 0
    total_conv_out_read = 0
    for d in [1, 2, 3]:
        K_d = int((model.dilations == d).sum().item())
        if K_d > 0:
            conv_in_read += B * C * T * bytes_per_elem
            total_conv_out_write += B * K_d * T * bytes_per_elem
            total_conv_out_read += 2 * B * K_d * T * bytes_per_elem

    features_write = B * (model.num_kernels * 2) * bytes_per_elem
    features_read = B * (model.num_kernels * 2) * bytes_per_elem

    activation_read = conv_in_read + total_conv_out_read + features_read
    activation_write = transposed_write + total_conv_out_write + features_write

    output_write = B * model.fc.out_features * bytes_per_elem

    return {
        'input_read': input_read,
        'param_read': param_read,
        'activation_read': activation_read,
        'activation_write': activation_write,
        'output_write': output_write
    }

def count_tst_bytes(model, input_shape):
    BYTES = 4
    B, T, C = input_shape

    d = model.embedding.out_features
    first = next(m for m in model.modules() if isinstance(m, nn.TransformerEncoderLayer))
    H = first.self_attn.num_heads
    ff_dim = first.linear1.out_features
    n_layers = len(model.transformer.layers)

    input_read = B * T * C * BYTES
    param_read = sum(p.numel() for p in model.parameters()) * BYTES

    act_read = 0
    act_write = 0

    act_write += B * T * d * BYTES

    for _ in range(n_layers):
        act_read += B * T * d * BYTES
        act_write += B * T * d * BYTES

        act_write += 3 * B * T * d * BYTES
        act_read += 3 * B * T * d * BYTES

        act_write += B * H * T * T * BYTES
        act_read += B * H * T * T * BYTES

        act_write += B * T * d * BYTES
        act_read += B * T * d * BYTES

        act_read += B * T * d * BYTES
        act_write += B * T * d * BYTES

        act_read += B * T * d * BYTES
        act_write += B * T * ff_dim * BYTES
        act_read += B * T * ff_dim * BYTES
        act_write += B * T * d * BYTES

    act_read += B * T * d * BYTES
    act_write += B * d * BYTES

    act_read += B * d * BYTES

    output_write = B * model.fc.out_features * BYTES

    return {
        'input_read': input_read,
        'param_read': param_read,
        'activation_read': act_read,
        'activation_write': act_write,
        'output_write': output_write
    }

test_x = np.load('data/test_x.npy')
batch_size = 1
test_batch = test_x[:batch_size]

B, seq_len, n_channels = test_batch.shape
n_classes = 6

models = {
    'TST': TransformerModel(n_channels, n_classes).to(device),
    'MiniROCKET': MiniROCKET(n_channels, n_classes).to(device),
    'MultiROCKET': MultiROCKET(n_channels, n_classes).to(device),
    'InceptionTime': InceptionTime(n_channels, n_classes).to(device)
}

for name in models:
    models[name].load_state_dict(torch.load(f'models/{name.lower()}.pt', map_location=device))
    models[name].eval()

results = []

print("=" * 70)
print("BYTES CALCULATION (Memory Read/Write per Inference)")
print("=" * 70)

model_counters = {
    'TST': count_tst_bytes,
    'MiniROCKET': count_minirocket_bytes,
    'MultiROCKET': count_multirocket_bytes,
    'InceptionTime': lambda m, shp: count_model_bytes(m, shp, add_initial_transpose_write=True)
}

for model_name, model in models.items():
    print(f"\n[{model_name}]")

    if model_name in model_counters:
        byte_counts = model_counters[model_name](model, test_batch.shape)
    else:
        byte_counts = count_model_bytes(model, test_batch.shape)

    total_read = byte_counts['input_read'] + byte_counts['param_read'] + byte_counts['activation_read']
    total_write = byte_counts['activation_write'] + byte_counts['output_write']
    total_bytes = total_read + total_write

    print(f"  Input read:        {byte_counts['input_read']:>12,} bytes")
    print(f"  Param read:        {byte_counts['param_read']:>12,} bytes")
    print(f"  Activation read:   {byte_counts['activation_read']:>12,} bytes")
    print(f"  Activation write:  {byte_counts['activation_write']:>12,} bytes")
    print(f"  Output write:      {byte_counts['output_write']:>12,} bytes")
    print(f"  Total read:        {total_read:>12,} bytes")
    print(f"  Total write:       {total_write:>12,} bytes")
    print(f"  Total bytes:       {total_bytes:>12,} bytes")

    method = 'hook' if model_name == 'InceptionTime' else 'analytical'

    results.append({
        'model': model_name,
        'input_read_bytes': byte_counts['input_read'],
        'param_read_bytes': byte_counts['param_read'],
        'activation_read_bytes': byte_counts['activation_read'],
        'activation_write_bytes': byte_counts['activation_write'],
        'output_write_bytes': byte_counts['output_write'],
        'total_read_bytes': total_read,
        'total_write_bytes': total_write,
        'total_bytes': total_bytes,
        'method': method
    })

print(f"\n[KNN]")
d = seq_len * n_channels
train_x = np.load('data/train_x.npy', mmap_mode='r')
n_train = len(train_x)
bytes_per_elem = 4

input_read = d * bytes_per_elem
train_data_read = n_train * d * bytes_per_elem
distances_write = n_train * bytes_per_elem
k = 5
topk_read = n_train * bytes_per_elem
topk_write = k * (bytes_per_elem + 4)
output_write = n_classes * bytes_per_elem

total_read = input_read + train_data_read + topk_read
total_write = distances_write + topk_write + output_write
total_bytes = total_read + total_write

print(f"  Input read:        {input_read:>12,} bytes")
print(f"  Param read:        {0:>12,} bytes")
print(f"  Activation read:   {train_data_read + topk_read:>12,} bytes")
print(f"  Activation write:  {distances_write + topk_write:>12,} bytes")
print(f"  Output write:      {output_write:>12,} bytes")
print(f"  Total read:        {total_read:>12,} bytes")
print(f"  Total write:       {total_write:>12,} bytes")
print(f"  Total bytes:       {total_bytes:>12,} bytes")

results.append({
    'model': 'KNN',
    'input_read_bytes': input_read,
    'param_read_bytes': 0,
    'activation_read_bytes': train_data_read + topk_read,
    'activation_write_bytes': distances_write + topk_write,
    'output_write_bytes': output_write,
    'total_read_bytes': total_read,
    'total_write_bytes': total_write,
    'total_bytes': total_bytes,
    'method': 'analytical'
})

print(f"\n[RandomForest]")

with open('models/rf_cuml.pkl', 'rb') as f:
    rf_model = pickle.load(f)

try:
    n_trees = getattr(rf_model, "n_estimators", getattr(rf_model, "n_estimators_", 100))
except:
    n_trees = 100

avg_depth = 15
nodes_per_tree = 2 * avg_depth
bytes_per_node = 16

input_read = d * bytes_per_elem
tree_nodes_read = n_trees * nodes_per_tree * bytes_per_node
path_write = n_trees * avg_depth * 4
votes_write = n_trees * bytes_per_elem
output_write = n_classes * bytes_per_elem

total_read = input_read + tree_nodes_read
total_write = path_write + votes_write + output_write
total_bytes = total_read + total_write

print(f"  Input read:        {input_read:>12,} bytes")
print(f"  Param read:        {tree_nodes_read:>12,} bytes")
print(f"  Activation read:   {0:>12,} bytes")
print(f"  Activation write:  {path_write + votes_write:>12,} bytes")
print(f"  Output write:      {output_write:>12,} bytes")
print(f"  Total read:        {total_read:>12,} bytes")
print(f"  Total write:       {total_write:>12,} bytes")
print(f"  Total bytes:       {total_bytes:>12,} bytes")

results.append({
    'model': 'RandomForest',
    'input_read_bytes': input_read,
    'param_read_bytes': tree_nodes_read,
    'activation_read_bytes': 0,
    'activation_write_bytes': path_write + votes_write,
    'output_write_bytes': output_write,
    'total_read_bytes': total_read,
    'total_write_bytes': total_write,
    'total_bytes': total_bytes,
    'method': 'analytical'
})

df = pd.DataFrame(results)
df.to_csv('logs/bytes_per_inf.csv', index=False)

print("\n" + "=" * 70)
print("BYTES SUMMARY (per inference)")
print("=" * 70)
print(f"{'Model':<20} {'Total Read':<15} {'Total Write':<15} {'Total':<15}")
print("-" * 70)
for _, row in df.iterrows():
    print(f"{row['model']:<20} {row['total_read_bytes']:>14,} {row['total_write_bytes']:>14,} {row['total_bytes']:>14,}")

print("\n" + "=" * 70)
print("CALCULATION METHODOLOGY")
print("=" * 70)
print(f"""
UNIT: Bytes (float32 = 4 bytes/element)
Batch size: {batch_size} (single inference)

BREAKDOWN CATEGORIES:
  Input read:       Raw input tensor read once
  Param read:       Model weights/kernels read once
  Activation read:  Intermediate tensors read (for next layer/operation)
  Activation write: Intermediate tensors written
  Output write:     Final prediction written

Deep Learning Models:
  TST:
    - Uses model's actual hyperparameters (d={models['TST'].embedding.out_features},
      H={next(m for m in models['TST'].modules() if isinstance(m, nn.TransformerEncoderLayer)).self_attn.num_heads},
      n_layers={len(models['TST'].transformer.layers)})
    - Params: embedding + 2×(MHA in_proj + out_proj + FFN) + final FC
    - Activations: Embedding output, LayerNorm (2 per layer), Q/K/V projections,
      attention scores, attention output, FFN intermediates, residual connections
    - Each tensor explicitly counted for read and write separately

  MiniROCKET:
    - Params: {1000} conv kernels (9×{n_channels}) + FC weights
    - Activation flow:
      * Transpose writes (B×C×T×4)
      * Conv reads transposed input (B×C×T×4)
      * Conv writes output (B×K×T×4)
      * PPV and max each read conv output (2×B×K×T×4)
      * Features concatenated and read by FC (B×2K×4)
    - Functional ops (F.conv1d) explicitly counted

  MultiROCKET:
    - Params: {2000} conv kernels + FC weights
    - Activation flow:
      * Each dilation group reads transposed input independently
      * 3 dilation groups × input read (B×C×T×4 each)
      * Per-group conv outputs and feature pooling
    - Functional ops explicitly counted

  InceptionTime:
    - Params: All conv/BN/FC weights in Inception modules
    - Activations: Multi-branch conv outputs, pooling, GAP
    - Standard module hooks

KNN (analytical):
  Input read:       {d} features × 4 bytes
  Activation read:  Training data ({n_train}×{d}×4 B) + distance array for top-k
  Activation write: {n_train} distances + top-{k} indices/distances
  Output write:     {n_classes} class probabilities

  Note: Training data classified as activation (not parameter) since it's
        accessed during inference like intermediate computations
  Assumption: Brute-force all-pairs distance computation
  Alternative: Indexed search (KD-tree, Ball-tree) would reduce train data read

RandomForest (analytical):
  Input read:       {d} features × 4 bytes
  Param read:       {n_trees} trees × {nodes_per_tree} nodes × {bytes_per_node} B/node
  Activation write: Path traces + per-tree votes
  Output write:     {n_classes} class probabilities

  Node structure:   16 bytes (conservative: feature_idx + threshold + leaf_value + alignment)
                    Range: 12-16B depending on implementation (sklearn/cuML/Treelite)
  Assumption:       avg_depth ≈ {avg_depth}, nodes_per_tree ≈ 2×avg_depth
  Note:             Actual implementation may have pointers/overhead

ASSUMPTIONS:
  - Single inference (batch=1)
  - FP32 precision (4 bytes per element)
  - Each tensor read once and written once per use
  - No gradient computation (inference only)
  - No memory reuse/in-place operations explicitly counted
  - ROCKET models: transpose creates new tensor (contiguous copy)
  - TST: Includes LayerNorm overhead (2 per transformer layer)
""")

print("=" * 70)
print("✓ Bytes statistics saved to logs/bytes_per_inf.csv")
print("✓ Separate columns: input_read, param_read, activation_read/write, output_write")
print("=" * 70)

BYTES CALCULATION (Memory Read/Write per Inference)

[TST]
  Input read:               4,608 bytes
  Param read:             403,992 bytes
  Activation read:      1,278,208 bytes
  Activation write:     1,278,208 bytes
  Output write:                24 bytes
  Total read:           1,686,808 bytes
  Total write:          1,278,232 bytes
  Total bytes:          2,965,040 bytes

[MiniROCKET]
  Input read:               4,608 bytes
  Param read:             372,024 bytes
  Activation read:      1,036,608 bytes
  Activation write:       524,608 bytes
  Output write:                24 bytes
  Total read:           1,413,240 bytes
  Total write:            524,632 bytes
  Total bytes:          1,937,872 bytes

[MultiROCKET]
  Input read:               4,608 bytes
  Param read:             744,024 bytes
  Activation read:      2,077,824 bytes
  Activation write:     1,044,608 bytes
  Output write:                24 bytes
  Total read:           2,826,456 bytes
  Total write:          1,044,63

step 6

In [24]:
import os
import time
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import pynvml
from scipy import stats
from abc import ABC, abstractmethod
import pickle
import cupy as cp

os.makedirs('calibration', exist_ok=True)

pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)

try:
    _ = pynvml.nvmlDeviceGetTotalEnergyConsumption(handle)
    energy_counter_supported = True
    method = "energy_counter"
except pynvml.NVMLError:
    energy_counter_supported = False
    method = "power_sampling"

device = torch.device('cuda')

def _sync_all():
    try:
        cp.cuda.runtime.deviceSynchronize()
    except Exception:
        pass
    torch.cuda.synchronize()

class InferenceAdapter(ABC):
    def __init__(self, model_name):
        self.model_name = model_name
        self.model = None
        self.warmed_up = False
        self.expected_shape = None

    def _validate(self, X, expect_channels=None, dtype=np.float32):
        X = np.asarray(X)
        if X.ndim != 3:
            raise ValueError(f"Expected shape [B,T,C], got {X.shape}")
        if self.expected_shape is not None and tuple(X.shape) != self.expected_shape:
            raise ValueError(f"Shape mismatch: expected {self.expected_shape}, got {X.shape}")
        if expect_channels is not None and X.shape[2] != expect_channels:
            raise ValueError(f"Channel mismatch: expected {expect_channels}, got {X.shape[2]}")
        if X.dtype != dtype:
            X = X.astype(dtype, copy=False)
        return X

    @abstractmethod
    def load_model(self):
        pass

    @abstractmethod
    def infer_one_batch(self, X):
        pass

    def warmup(self, input_shape, n_warmup=20):
        self.expected_shape = tuple(input_shape)
        dummy_input = np.random.randn(*self.expected_shape).astype(np.float32)
        for _ in range(n_warmup):
            _ = self.infer_one_batch(dummy_input)
        self.warmed_up = True

class KNNAdapter(InferenceAdapter):
    def __init__(self, n_channels=9):
        super().__init__("KNN")
        self.n_channels = n_channels
        self.expected_flat = None
        self.load_model()

    def load_model(self):
        with open('models/knn_cuml.pkl', 'rb') as f:
            self.model = pickle.load(f)
        self.expected_flat = getattr(self.model, "n_features_in_", None) or getattr(self.model, "n_cols", None)

    def infer_one_batch(self, X):
        X = self._validate(X, self.n_channels)
        B, T, C = X.shape
        if self.expected_flat is not None and T * C != self.expected_flat:
            raise ValueError(f"Flattened features {T*C} != model expects {self.expected_flat}")
        X_flat = X.reshape(B, -1)
        with cp.cuda.Device(0):
            X_cp = cp.asarray(X_flat)
            probs_cp = self.model.predict_proba(X_cp)
        cp.cuda.runtime.deviceSynchronize()
        probs = cp.asnumpy(probs_cp)
        return probs.astype(np.float32, copy=False)

class RandomForestAdapter(InferenceAdapter):
    def __init__(self, n_channels=9):
        super().__init__("RandomForest")
        self.n_channels = n_channels
        self.expected_flat = None
        self.load_model()

    def load_model(self):
        with open('models/rf_cuml.pkl', 'rb') as f:
            self.model = pickle.load(f)
        self.expected_flat = getattr(self.model, "n_features_in_", None) or getattr(self.model, "n_cols", None)

    def infer_one_batch(self, X):
        X = self._validate(X, self.n_channels)
        B, T, C = X.shape
        if self.expected_flat is not None and T * C != self.expected_flat:
            raise ValueError(f"Flattened features {T*C} != model expects {self.expected_flat}")
        X_flat = X.reshape(B, -1)
        with cp.cuda.Device(0):
            X_cp = cp.asarray(X_flat)
            probs_cp = self.model.predict_proba(X_cp)
        cp.cuda.runtime.deviceSynchronize()
        probs = cp.asnumpy(probs_cp)
        return probs.astype(np.float32, copy=False)

class TransformerModel(nn.Module):
    def __init__(self, input_dim, num_classes, d_model=64, nhead=4, num_layers=2):
        super().__init__()
        self.embedding = nn.Linear(input_dim, d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead,
                                                     dim_feedforward=256, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(d_model, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = self.transformer(x)
        x = x.mean(dim=1)
        return self.fc(x)

class TSTAdapter(InferenceAdapter):
    def __init__(self, n_channels=9, n_classes=6):
        super().__init__("TST")
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.load_model()

    def load_model(self):
        self.model = TransformerModel(self.n_channels, self.n_classes).to(device)
        self.model.load_state_dict(torch.load('models/tst.pt', map_location=device))
        self.model.eval()

    def infer_one_batch(self, X):
        X = self._validate(X, self.n_channels)
        X_tensor = torch.FloatTensor(X).to(device)
        with torch.inference_mode():
            logits = self.model(X_tensor)
            probs = torch.softmax(logits, dim=1)
        torch.cuda.synchronize()
        return probs.detach().cpu().numpy().astype(np.float32, copy=False)

class MiniROCKET(nn.Module):
    def __init__(self, input_channels, num_classes, num_kernels=1000):
        super().__init__()
        self.kernels = nn.Parameter(torch.randn(num_kernels, input_channels, 9))
        self.fc = nn.Linear(num_kernels * 2, num_classes)

    def forward(self, x):
        x = x.transpose(1, 2).contiguous()
        conv_out = torch.nn.functional.conv1d(x, self.kernels, padding=4)
        ppv = (conv_out > 0).float().mean(dim=2)
        mx = conv_out.amax(dim=2)
        features = torch.cat([ppv, mx], dim=1)
        return self.fc(features)

class MiniROCKETAdapter(InferenceAdapter):
    def __init__(self, n_channels=9, n_classes=6):
        super().__init__("MiniROCKET")
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.load_model()

    def load_model(self):
        self.model = MiniROCKET(self.n_channels, self.n_classes).to(device)
        self.model.load_state_dict(torch.load('models/minirocket.pt', map_location=device))
        self.model.eval()

    def infer_one_batch(self, X):
        X = self._validate(X, self.n_channels)
        X_tensor = torch.FloatTensor(X).to(device)
        with torch.inference_mode():
            logits = self.model(X_tensor)
            probs = torch.softmax(logits, dim=1)
        torch.cuda.synchronize()
        return probs.detach().cpu().numpy().astype(np.float32, copy=False)

class MultiROCKET(nn.Module):
    def __init__(self, input_channels, num_classes, num_kernels=2000):
        super().__init__()
        self.num_kernels = num_kernels
        self.kernels = nn.Parameter(torch.randn(num_kernels, input_channels, 9))
        dilations = torch.randint(1, 4, (num_kernels,))
        self.register_buffer('dilations', dilations)
        self.fc = nn.Linear(num_kernels * 2, num_classes)

    def forward(self, x):
        x = x.transpose(1, 2).contiguous()
        features_list = []
        for d in [1, 2, 3]:
            mask = self.dilations == d
            if mask.any():
                kernels_d = self.kernels[mask]
                conv_out = torch.nn.functional.conv1d(x, kernels_d, padding=4*d, dilation=d)
                ppv = (conv_out > 0).float().mean(dim=2)
                mx = conv_out.amax(dim=2)
                features_list.append(torch.cat([ppv, mx], dim=1))
        features = torch.cat(features_list, dim=1)
        return self.fc(features)

class MultiROCKETAdapter(InferenceAdapter):
    def __init__(self, n_channels=9, n_classes=6):
        super().__init__("MultiROCKET")
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.load_model()

    def load_model(self):
        self.model = MultiROCKET(self.n_channels, self.n_classes).to(device)
        self.model.load_state_dict(torch.load('models/multirocket.pt', map_location=device))
        self.model.eval()

    def infer_one_batch(self, X):
        X = self._validate(X, self.n_channels)
        X_tensor = torch.FloatTensor(X).to(device)
        with torch.inference_mode():
            logits = self.model(X_tensor)
            probs = torch.softmax(logits, dim=1)
        torch.cuda.synchronize()
        return probs.detach().cpu().numpy().astype(np.float32, copy=False)

class InceptionModule(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=1)
        self.conv3 = nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1)
        self.conv5 = nn.Conv1d(in_channels, out_channels, kernel_size=5, padding=2)
        self.pool = nn.Sequential(nn.MaxPool1d(3, stride=1, padding=1),
                                  nn.Conv1d(in_channels, out_channels, kernel_size=1))
        self.bn = nn.BatchNorm1d(out_channels * 4)

    def forward(self, x):
        x1 = self.conv1(x)
        x2 = self.conv3(x)
        x3 = self.conv5(x)
        x4 = self.pool(x)
        out = torch.cat([x1, x2, x3, x4], dim=1)
        return torch.relu(self.bn(out))

class InceptionTime(nn.Module):
    def __init__(self, input_channels, num_classes):
        super().__init__()
        self.inception1 = InceptionModule(input_channels, 32)
        self.inception2 = InceptionModule(128, 64)
        self.gap = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(256, num_classes)

    def forward(self, x):
        x = x.transpose(1, 2).contiguous()
        x = self.inception1(x)
        x = self.inception2(x)
        x = self.gap(x).squeeze(-1)
        return self.fc(x)

class InceptionTimeAdapter(InferenceAdapter):
    def __init__(self, n_channels=9, n_classes=6):
        super().__init__("InceptionTime")
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.load_model()

    def load_model(self):
        self.model = InceptionTime(self.n_channels, self.n_classes).to(device)
        self.model.load_state_dict(torch.load('models/inceptiontime.pt', map_location=device))
        self.model.eval()

    def infer_one_batch(self, X):
        X = self._validate(X, self.n_channels)
        X_tensor = torch.FloatTensor(X).to(device)
        with torch.inference_mode():
            logits = self.model(X_tensor)
            probs = torch.softmax(logits, dim=1)
        torch.cuda.synchronize()
        return probs.detach().cpu().numpy().astype(np.float32, copy=False)

def measure_baseline(duration_sec=10, sample_interval_ms=100):
    samples = []
    start = time.time()
    interval_sec = sample_interval_ms / 1000.0

    if energy_counter_supported:
        e0 = pynvml.nvmlDeviceGetTotalEnergyConsumption(handle)
        while time.time() - start < duration_sec:
            time.sleep(interval_sec)
            e1 = pynvml.nvmlDeviceGetTotalEnergyConsumption(handle)
            samples.append(e1 - e0)
            e0 = e1
    else:
        while time.time() - start < duration_sec:
            time.sleep(interval_sec)
            power = pynvml.nvmlDeviceGetPowerUsage(handle) / 1000.0
            samples.append(power * interval_sec * 1000)

    elapsed = time.time() - start
    total_mj = sum(samples)
    baseline_mj_per_sec = total_mj / elapsed
    return baseline_mj_per_sec

def measure_energy_counter(adapter, X_batch, n_inferences):
    _sync_all()
    t0 = time.perf_counter()
    e_start = pynvml.nvmlDeviceGetTotalEnergyConsumption(handle)

    for _ in range(n_inferences):
        _ = adapter.infer_one_batch(X_batch)

    _sync_all()
    e_end = pynvml.nvmlDeviceGetTotalEnergyConsumption(handle)
    t1 = time.perf_counter()

    energy_mj = e_end - e_start
    latency_ms = (t1 - t0) * 1000.0 / n_inferences

    return energy_mj, latency_ms

def measure_power_sampling(adapter, X_batch, n_inferences, sample_interval_ms=10):
    power_samples = []
    timestamps = []

    def sample_power():
        while getattr(sample_power, 'running', True):
            t = time.perf_counter()
            p = pynvml.nvmlDeviceGetPowerUsage(handle) / 1000.0
            power_samples.append(p)
            timestamps.append(t)
            time.sleep(sample_interval_ms / 1000.0)

    import threading
    sample_power.running = True
    sampler_thread = threading.Thread(target=sample_power, daemon=True)
    sampler_thread.start()

    _sync_all()
    t_start = time.perf_counter()

    for _ in range(n_inferences):
        _ = adapter.infer_one_batch(X_batch)

    _sync_all()
    t_end = time.perf_counter()

    sample_power.running = False
    sampler_thread.join(timeout=1)

    if len(power_samples) < 2 or len(timestamps) < 2:
        energy_mj = 0
    else:
        timestamps = np.array(timestamps)
        power_samples = np.array(power_samples)
        mask = (timestamps >= t_start) & (timestamps <= t_end)
        if mask.sum() >= 2:
            energy_j = np.trapz(power_samples[mask], timestamps[mask])
            energy_mj = energy_j * 1000
        else:
            energy_mj = np.mean(power_samples) * (t_end - t_start) * 1000

    latency_ms = (t_end - t_start) * 1000.0 / n_inferences

    return energy_mj, latency_ms

test_x = np.load('data/test_x.npy')
batch_size = 1
test_batch = test_x[:batch_size]

adapters = {
    'KNN': KNNAdapter(),
    'RandomForest': RandomForestAdapter(),
    'TST': TSTAdapter(),
    'MiniROCKET': MiniROCKETAdapter(),
    'MultiROCKET': MultiROCKETAdapter(),
    'InceptionTime': InceptionTimeAdapter()
}

n_warmup = 20
n_rounds = 5
target_duration_sec = 10.0

print("=" * 70)
print("GPU ENERGY CALIBRATION")
print("=" * 70)
print(f"Method: {method}")
print(f"Energy counter supported: {energy_counter_supported}")
print(f"Warmup: {n_warmup} inferences")
print(f"Target duration: {target_duration_sec:.1f}s/round")
print(f"Rounds: {n_rounds}")

results = []

for name, adapter in adapters.items():
    print(f"\n[{name}]")

    print("  Warming up...")
    adapter.warmup(test_batch.shape, n_warmup=n_warmup)

    print("  Measuring baseline (pre)...")
    baseline_pre_mj_per_sec = measure_baseline(duration_sec=5, sample_interval_ms=50)
    print(f"  Baseline (pre): {baseline_pre_mj_per_sec:.2f} mJ/s")

    print("  Probing latency...")
    if energy_counter_supported:
        _, probe_lat = measure_energy_counter(adapter, test_batch, 50)
    else:
        _, probe_lat = measure_power_sampling(adapter, test_batch, 50)

    n_inferences_per_run = max(200, int(np.ceil(target_duration_sec / (probe_lat / 1000.0))))
    n_inferences_per_run = min(n_inferences_per_run, 1000)

    print(f"  Latency: {probe_lat:.3f} ms/inf → using {n_inferences_per_run} inferences × {n_rounds} rounds")

    round_E_total_mj = []
    round_duration_s = []
    round_latencies_ms = []

    for r in range(n_rounds):
        if energy_counter_supported:
            energy_total_mj, latency_ms = measure_energy_counter(adapter, test_batch, n_inferences_per_run)
        else:
            energy_total_mj, latency_ms = measure_power_sampling(adapter, test_batch, n_inferences_per_run)

        duration_sec = (latency_ms / 1000.0) * n_inferences_per_run
        round_E_total_mj.append(energy_total_mj)
        round_duration_s.append(duration_sec)
        round_latencies_ms.append(latency_ms)

        print(f"    Round {r+1}: Total={energy_total_mj:.2f} mJ, Lat={latency_ms:.3f} ms/inf")
        time.sleep(0.5)

    print("  Measuring baseline (post)...")
    baseline_post_mj_per_sec = measure_baseline(duration_sec=5, sample_interval_ms=50)
    baseline_mj_per_sec = 0.5 * (baseline_pre_mj_per_sec + baseline_post_mj_per_sec)
    print(f"  Baseline (post): {baseline_post_mj_per_sec:.2f} mJ/s → Used={baseline_mj_per_sec:.2f} mJ/s")

    round_E_total_mj = np.array(round_E_total_mj)
    round_duration_s = np.array(round_duration_s)
    round_latencies_ms = np.array(round_latencies_ms)

    energy_total_per_inf = round_E_total_mj / n_inferences_per_run
    energy_net_per_inf = (round_E_total_mj - baseline_mj_per_sec * round_duration_s) / n_inferences_per_run

    mean_energy_total = energy_total_per_inf.mean()
    std_energy_total = energy_total_per_inf.std(ddof=1)
    sem_energy_total = std_energy_total / np.sqrt(n_rounds)
    ci95_energy_total = sem_energy_total * stats.t.ppf(0.975, n_rounds - 1)

    mean_energy_net = energy_net_per_inf.mean()
    std_energy_net = energy_net_per_inf.std(ddof=1)
    sem_energy_net = std_energy_net / np.sqrt(n_rounds)
    ci95_energy_net = sem_energy_net * stats.t.ppf(0.975, n_rounds - 1)

    mean_latency = round_latencies_ms.mean()
    std_latency = round_latencies_ms.std(ddof=1)
    sem_latency = std_latency / np.sqrt(n_rounds)
    ci95_latency = sem_latency * stats.t.ppf(0.975, n_rounds - 1)

    results.append({
        'model': name,
        'mean_energy_total_mj_per_inf': mean_energy_total,
        'std_energy_total_mj_per_inf': std_energy_total,
        'ci95_energy_total_mj_per_inf': ci95_energy_total,
        'mean_energy_net_mj_per_inf': mean_energy_net,
        'std_energy_net_mj_per_inf': std_energy_net,
        'ci95_energy_net_mj_per_inf': ci95_energy_net,
        'mean_latency_ms_per_inf': mean_latency,
        'std_latency_ms_per_inf': std_latency,
        'ci95_latency_ms_per_inf': ci95_latency,
        'method': method,
        'baseline_subtracted_per_round': True,
        'n_rounds': n_rounds,
        'n_inferences_per_round': n_inferences_per_run,
        'baseline_mj_per_sec': baseline_mj_per_sec,
        'baseline_pre_mj_per_sec': baseline_pre_mj_per_sec,
        'baseline_post_mj_per_sec': baseline_post_mj_per_sec
    })

df = pd.DataFrame(results)
df.to_csv('calibration/calibration_measurements.csv', index=False)

print("\n" + "=" * 70)
print("CALIBRATION SUMMARY")
print("=" * 70)
print(f"{'Model':<20} {'E_net (mJ/inf)':<25} {'E_total (mJ/inf)':<25} {'Latency (ms/inf)':<25}")
print(f"{'':20} {'Mean ± 95%CI':<25} {'Mean ± 95%CI':<25} {'Mean ± 95%CI':<25}")
print("-" * 70)
for _, row in df.iterrows():
    net_str = f"{row['mean_energy_net_mj_per_inf']:.3f} ± {row['ci95_energy_net_mj_per_inf']:.3f}"
    total_str = f"{row['mean_energy_total_mj_per_inf']:.3f} ± {row['ci95_energy_total_mj_per_inf']:.3f}"
    latency_str = f"{row['mean_latency_ms_per_inf']:.3f} ± {row['ci95_latency_ms_per_inf']:.3f}"
    print(f"{row['model']:<20} {net_str:<25} {total_str:<25} {latency_str:<25}")

print("\n" + "=" * 70)
print("MEASUREMENT DETAILS")
print("=" * 70)
print(f"Method: {method}")
print("  - Wall-clock + full device synchronization (CuPy + PyTorch)")
if method == "energy_counter":
    print("  - Energy counter read at start/end")
else:
    print("  - Power sampling with trapz integration")
print(f"Baseline: Sandwich method (pre + post, per-model)")
print(f"Adaptive inferences: 200-1000 per round")
print(f"Target duration: {target_duration_sec:.1f}s/round (all models)")
print(f"Rounds: {n_rounds} (all models, 0.5s rest between rounds)")
print(f"Confidence interval: 95%")
print(f"Units: mJ per inference, ms per inference")
print("\nE_net = (E_total - baseline × duration) / n_inferences")
print("E_total = raw energy consumption without baseline subtraction")
print("\n" + "=" * 70)
print("✓ Calibration measurements saved to calibration/calibration_measurements.csv")
print("=" * 70)

pynvml.nvmlShutdown()

GPU ENERGY CALIBRATION
Method: energy_counter
Energy counter supported: True
Warmup: 20 inferences
Target duration: 10.0s/round
Rounds: 5

[KNN]
  Warming up...
  Measuring baseline (pre)...
  Baseline (pre): 31602.84 mJ/s
  Probing latency...
  Latency: 3.149 ms/inf → using 1000 inferences × 5 rounds
    Round 1: Total=104885.00 mJ, Lat=3.105 ms/inf
    Round 2: Total=104841.00 mJ, Lat=3.033 ms/inf
    Round 3: Total=101600.00 mJ, Lat=3.028 ms/inf
    Round 4: Total=105004.00 mJ, Lat=3.068 ms/inf
    Round 5: Total=101995.00 mJ, Lat=2.997 ms/inf
  Measuring baseline (post)...
  Baseline (post): 30948.47 mJ/s → Used=31275.66 mJ/s

[RandomForest]
  Warming up...
  Measuring baseline (pre)...
  Baseline (pre): 30938.82 mJ/s
  Probing latency...
  Latency: 18.822 ms/inf → using 532 inferences × 5 rounds
    Round 1: Total=308013.00 mJ, Lat=18.564 ms/inf
    Round 2: Total=307735.00 mJ, Lat=18.629 ms/inf
    Round 3: Total=308752.00 mJ, Lat=18.530 ms/inf
    Round 4: Total=306014.00 mJ, La

step 7

In [25]:
import os
import json
import numpy as np
import pandas as pd

os.makedirs('logs', exist_ok=True)

macs_df = pd.read_csv('logs/macs_per_inf.csv')
bytes_df = pd.read_csv('logs/bytes_per_inf.csv')
energy_df = pd.read_csv('calibration/calibration_measurements.csv')

macs_dict = dict(zip(macs_df['model'], macs_df['macs_per_inf']))
bytes_dict = dict(zip(bytes_df['model'], bytes_df['total_bytes']))
energy_dict = dict(zip(energy_df['model'], energy_df['mean_energy_net_mj_per_inf']))
energy_ci_dict = dict(zip(energy_df['model'], energy_df['ci95_energy_net_mj_per_inf']))
latency_dict = dict(zip(energy_df['model'], energy_df['mean_latency_ms_per_inf']))
latency_ci_dict = dict(zip(energy_df['model'], energy_df['ci95_latency_ms_per_inf']))

all_models = set(macs_dict.keys()) & set(bytes_dict.keys()) & set(energy_dict.keys())

full_data = []
for model in all_models:
    flops = macs_dict[model] * 2
    bytes_total = bytes_dict[model]
    energy = energy_dict[model]
    energy_ci = energy_ci_dict[model]
    latency = latency_dict[model]
    latency_ci = latency_ci_dict[model]

    full_data.append({
        'model': model,
        'flops_per_inf': flops,
        'bytes_per_inf': bytes_total,
        'energy_mj_per_inf': energy,
        'energy_ci95_mj_per_inf': energy_ci,
        'latency_ms_per_inf': latency,
        'latency_ci95_ms_per_inf': latency_ci,
        'flops_bytes_ratio': flops / bytes_total if bytes_total > 0 else 0
    })

full_df = pd.DataFrame(full_data)
full_df = full_df.sort_values('flops_bytes_ratio')

calibration_df = full_df.copy()

print("=" * 70)
print("CALIBRATION SAMPLE SELECTION")
print("=" * 70)
print(f"\nTotal models: {len(full_df)}")
print(f"Calibration set: {len(calibration_df)} models (all models used)")
print("\nModels (sorted by FLOPs/Bytes ratio):")
print("-" * 70)
print(f"{'Model':<20} {'FLOPs':<15} {'Bytes':<15} {'Ratio':<12} {'Energy (mJ)':<15}")
print("-" * 70)
for _, row in full_df.iterrows():
    print(f"{row['model']:<20} {row['flops_per_inf']:>14,.0f} {row['bytes_per_inf']:>14,.0f} "
          f"{row['flops_bytes_ratio']:>11.2f} {row['energy_mj_per_inf']:>14.3f}")

print("\n" + "=" * 70)
print("ALL MODELS DETAILS")
print("=" * 70)
for _, row in calibration_df.iterrows():
    print(f"\n[{row['model']}]")
    print(f"  FLOPs:           {row['flops_per_inf']:>14,.0f}")
    print(f"  Bytes:           {row['bytes_per_inf']:>14,.0f}")
    print(f"  FLOPs/Bytes:     {row['flops_bytes_ratio']:>14.2f}")
    print(f"  Energy:          {row['energy_mj_per_inf']:>14.3f} ± {row['energy_ci95_mj_per_inf']:.3f} mJ/inf")
    print(f"  Latency:         {row['latency_ms_per_inf']:>14.3f} ± {row['latency_ci95_ms_per_inf']:.3f} ms/inf")

ratio_range = calibration_df['flops_bytes_ratio'].max() / calibration_df['flops_bytes_ratio'].min()
flops_range = calibration_df['flops_per_inf'].max() / calibration_df['flops_per_inf'].min()
bytes_range = calibration_df['bytes_per_inf'].max() / calibration_df['bytes_per_inf'].min()

print("\n" + "=" * 70)
print("COVERAGE ANALYSIS")
print("=" * 70)
print(f"FLOPs/Bytes ratio range:  {ratio_range:.2f}x  "
      f"({calibration_df['flops_bytes_ratio'].min():.2f} → {calibration_df['flops_bytes_ratio'].max():.2f})")
print(f"FLOPs range:              {flops_range:.2f}x  "
      f"({calibration_df['flops_per_inf'].min():,.0f} → {calibration_df['flops_per_inf'].max():,.0f})")
print(f"Bytes range:              {bytes_range:.2f}x  "
      f"({calibration_df['bytes_per_inf'].min():,.0f} → {calibration_df['bytes_per_inf'].max():,.0f})")

os.makedirs('calibration', exist_ok=True)
calibration_df.to_csv('logs/calibration_df.csv', index=False)
calibration_df.to_csv('calibration/calibration_df.csv', index=False)

ratio_min = calibration_df['flops_bytes_ratio'].replace(0, np.nan).min()
ratio_max = calibration_df['flops_bytes_ratio'].max()
ratio_span = (ratio_max / ratio_min) if np.isfinite(ratio_min) and ratio_min > 0 else float('inf')

q30 = full_df['flops_bytes_ratio'].quantile(0.30)
q70 = full_df['flops_bytes_ratio'].quantile(0.70)
covers_low = ratio_min <= q30
covers_high = ratio_max >= q70

coverage_report = {
    "timestamp_utc": pd.Timestamp.utcnow().isoformat(),
    "selected_models": calibration_df['model'].tolist(),
    "n_models": len(calibration_df),
    "coverage": {
        "ratio_min": float(ratio_min),
        "ratio_max": float(ratio_max),
        "ratio_span_x": float(ratio_span),
        "flops_min": int(calibration_df['flops_per_inf'].min()),
        "flops_max": int(calibration_df['flops_per_inf'].max()),
        "bytes_min": int(calibration_df['bytes_per_inf'].min()),
        "bytes_max": int(calibration_df['bytes_per_inf'].max()),
        "q30_threshold": float(q30),
        "q70_threshold": float(q70),
        "covers_bytes_dominant": bool(covers_low),
        "covers_flops_dominant": bool(covers_high)
    },
    "quality_gate": {
        "ratio_span_min": 2.0,
        "min_models": 3,
        "requires_low_coverage": True,
        "requires_high_coverage": True
    },
    "note": "All-six model calibration (KNN, RF, TST, MiniROCKET, MultiROCKET, InceptionTime)"
}

with open('calibration/coverage.json', 'w') as f:
    json.dump(coverage_report, f, indent=2)

if ratio_span >= 2.0:
    print("\n✓ Good coverage: FLOPs/Bytes ratio spans ≥2x (FLOPs-dominant to Bytes-dominant)")
    if covers_low:
        print(f"✓ Covers Bytes-dominant region (ratio_min={ratio_min:.2f} ≤ Q30={q30:.2f})")
    if covers_high:
        print(f"✓ Covers FLOPs-dominant region (ratio_max={ratio_max:.2f} ≥ Q70={q70:.2f})")
else:
    print("\n⚠ Warning: FLOPs/Bytes ratio range <2x")

assert (ratio_span >= 2.0 and covers_low and covers_high and len(calibration_df) >= 3), (
    f"Step7 quality gate failed: ratio_span={ratio_span:.2f}x (need ≥2x), "
    f"covers_low={covers_low}, covers_high={covers_high}, n={len(calibration_df)} (need ≥3)"
)

print("\n" + "=" * 70)
print("✓ Calibration data saved to calibration/calibration_df.csv")
print("✓ Backup copy saved to logs/calibration_df.csv")
print("✓ Coverage report saved to calibration/coverage.json")
print(f"✓ Quality gate passed: {len(calibration_df)} models, {ratio_span:.2f}x span, both ends covered")
print("=" * 70)

CALIBRATION SAMPLE SELECTION

Total models: 6
Calibration set: 6 models (all models used)

Models (sorted by FLOPs/Bytes ratio):
----------------------------------------------------------------------
Model                FLOPs           Bytes           Ratio        Energy (mJ)    
----------------------------------------------------------------------
RandomForest                  6,000         59,032        0.10          3.805
KNN                      25,579,008     25,628,088        1.00          8.389
MiniROCKET               20,760,000      1,937,872       10.71          0.361
MultiROCKET              41,520,000      3,871,088       10.73          3.178
TST                      34,030,336      2,965,040       11.48          4.921
InceptionTime            21,908,480      1,503,024       14.58          2.907

ALL MODELS DETAILS

[RandomForest]
  FLOPs:                    6,000
  Bytes:                   59,032
  FLOPs/Bytes:               0.10
  Energy:                   3.805 ± 2.323

step 8

In [34]:
import os
import json
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from scipy import stats
from scipy.optimize import lsq_linear
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

os.makedirs('logs', exist_ok=True)

df = pd.read_csv('logs/calibration_df.csv')

X = df[['flops_per_inf', 'bytes_per_inf']].values
y = df['energy_mj_per_inf'].values

# 从 CI95 还原 SE（标准误），做方差平均
if 'n_rounds' in df.columns:
    R = df['n_rounds'].astype(int).values
else:
    R = np.full(len(df), 5, dtype=int)

ci95 = df['energy_ci95_mj_per_inf'].values
tcrit = stats.t.ppf(0.975, np.clip(R - 1, 1, None))
y_meas_SE = ci95 / tcrit
pooled_meas_var = np.mean(y_meas_SE ** 2)
pooled_meas_SE = np.sqrt(pooled_meas_var)

# 标准化
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)

# 加权最小二乘（WLS），权重基于 SE
w = 1.0 / np.maximum(y_meas_SE**2, 1e-12)
model = LinearRegression()
model.fit(X_scaled, y, sample_weight=w)

alpha_scaled, beta_scaled = model.coef_
gamma = model.intercept_

flops_mean, bytes_mean = scaler_X.mean_
flops_std, bytes_std = scaler_X.scale_

alpha = alpha_scaled / flops_std
beta = beta_scaled / bytes_std
gamma_unscaled = gamma - alpha * flops_mean - beta * bytes_mean

# 残差计算
y_pred = model.predict(X_scaled)
residuals = y - y_pred

n = len(df)
p = 3

# 未加权版本（有量纲 mJ）
SSE = np.sum(residuals**2)
sigma_resid_unw = np.sqrt(SSE / (n - p))
r2_unw = r2_score(y, y_pred)
mse_unw = mean_squared_error(y, y_pred)
rmse_unw = np.sqrt(mse_unw)

# 加权版本（用于WLS诊断）
SSE_w = np.sum(w * residuals**2)
sigma_resid_w = np.sqrt(SSE_w / max(n - p, 1))  # 无量纲（whitened）
rmse_w = np.sqrt(np.average(residuals**2, weights=w))

# 加权R²
def r2_weighted(y_true, y_pred, weights):
    ybar = np.average(y_true, weights=weights)
    ss_res = np.sum(weights * (y_true - y_pred)**2)
    ss_tot = np.sum(weights * (y_true - ybar)**2)
    return 1.0 - ss_res / np.maximum(ss_tot, 1e-12)

r2_w = r2_weighted(y, y_pred, w)

# 模型不确定度（使用加权RMSE，与WLS口径一致，单位 mJ）
sigma_model = rmse_w

# 总不确定度（方差相加，单位 mJ）
total_uncertainty = np.sqrt(sigma_model**2 + pooled_meas_var)

# Δ95：默认用t分布（更保守），同时保存z版本
t_crit_overall = stats.t.ppf(0.975, n - p)
delta_95_t = t_crit_overall * total_uncertainty
delta_95_z = 1.96 * total_uncertainty

# 非负约束 WLS
X_scaled_with_intercept = np.column_stack([X_scaled, np.ones(n)])
w_sqrt = np.sqrt(w)
X_weighted = X_scaled_with_intercept * w_sqrt[:, np.newaxis]
y_weighted = y * w_sqrt

result_nnls = lsq_linear(X_weighted, y_weighted, bounds=(0, np.inf), method='bvls')
coef_nnls_scaled = result_nnls.x

alpha_nnls_scaled, beta_nnls_scaled, gamma_nnls = coef_nnls_scaled
alpha_nnls = alpha_nnls_scaled / flops_std
beta_nnls = beta_nnls_scaled / bytes_std
gamma_nnls_unscaled = gamma_nnls - alpha_nnls * flops_mean - beta_nnls * bytes_mean

y_pred_nnls = X_scaled_with_intercept @ coef_nnls_scaled
r2_nnls_w = r2_weighted(y, y_pred_nnls, w)

results = []
for i, row in df.iterrows():
    e_point = y_pred[i]
    e_upper = e_point + delta_95_t
    e_point_nnls = y_pred_nnls[i]
    e_upper_nnls = e_point_nnls + delta_95_t
    results.append({
        'model': row['model'],
        'flops': row['flops_per_inf'],
        'bytes': row['bytes_per_inf'],
        'energy_measured_mj': row['energy_mj_per_inf'],
        'energy_predicted_mj': e_point,
        'energy_upper95_mj': e_upper,
        'energy_predicted_mj_nnls': e_point_nnls,
        'energy_upper95_mj_nnls': e_upper_nnls,
        'residual_mj': residuals[i]
    })

results_df = pd.DataFrame(results)
results_df.to_csv('logs/fit_results.csv', index=False)

coeffs = {
    'alpha_flops_coefficient': float(alpha),
    'beta_bytes_coefficient': float(beta),
    'gamma_intercept': float(gamma_unscaled),
    'delta_95_mj': float(delta_95_t),
    'delta_95_z_mj': float(delta_95_z),
    'upper_method': 't-distribution',
    'units': 'mJ/inf',
    'formula': 'E = alpha * FLOPs + beta * Bytes + gamma',
    'method': 'WLS (weighted least squares)',
    'weights': '1 / SE^2 (SE from CI95)'
}

with open('logs/alpha_beta_gamma.json', 'w') as f:
    json.dump(coeffs, f, indent=2)

coeffs_nnls = {
    'alpha_flops_coefficient': float(alpha_nnls),
    'beta_bytes_coefficient': float(beta_nnls),
    'gamma_intercept': float(gamma_nnls_unscaled),
    'delta_95_mj': float(delta_95_t),
    'delta_95_z_mj': float(delta_95_z),
    'upper_method': 't-distribution',
    'r_squared': float(r2_nnls_w),
    'units': 'mJ/inf',
    'formula': 'E = alpha * FLOPs + beta * Bytes + gamma',
    'method': 'WLS with non-negative constraint',
    'note': 'Enforces non-negativity in scaled parameterization; unscaled gamma may be negative'
}

with open('logs/alpha_beta_gamma_nnls.json', 'w') as f:
    json.dump(coeffs_nnls, f, indent=2)

diag = {
    'r_squared_weighted': float(r2_w),
    'r_squared_nnls_weighted': float(r2_nnls_w),
    'r_squared_unweighted': float(r2_unw),
    'mse_unweighted': float(mse_unw),
    'rmse_unweighted_mj': float(rmse_unw),
    'rmse_weighted_mj': float(rmse_w),
    'residual_sigma_unweighted_mj': float(sigma_resid_unw),
    'residual_sigma_weighted_unitless': float(sigma_resid_w),
    'model_uncertainty_mj': float(sigma_model),
    'pooled_measurement_SE_mj': float(pooled_meas_SE),
    'pooled_measurement_var_mj2': float(pooled_meas_var),
    'total_uncertainty_mj': float(total_uncertainty),
    'delta_95_t_mj': float(delta_95_t),
    'delta_95_z_mj': float(delta_95_z),
    'z_critical': 1.96,
    't_critical': float(t_crit_overall),
    'n_samples': n,
    'degrees_of_freedom': n - p,
    'formula_upper': 'E_upper = E_point + delta_95_t_mj',
    'note': 'Model uncertainty uses weighted RMSE (mJ) for WLS consistency; Primary upper bound uses t-distribution'
}

with open('logs/fit_diag.json', 'w') as f:
    json.dump(diag, f, indent=2)

plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.scatter(y_pred, y, alpha=0.6, label='WLS', s=50)
plt.scatter(y_pred_nnls, y, alpha=0.4, marker='^', label='NNLS', s=50)
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--', lw=2)
plt.xlabel('Predicted Energy (mJ)')
plt.ylabel('Measured Energy (mJ)')
plt.title(f'Prediction vs Measured (R²_w={r2_w:.3f})')
plt.grid(True, alpha=0.3)
plt.legend()

plt.subplot(1, 3, 2)
stats.probplot(residuals, dist="norm", plot=plt)
plt.title('Residuals Q-Q Plot')
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 3)
plt.hist(residuals, bins=10, alpha=0.7, edgecolor='black')
plt.xlabel('Residual (mJ)')
plt.ylabel('Frequency')
plt.title(f'Residuals (RMSE={rmse_unw:.3f} mJ)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('logs/fit_diagnostics.png', dpi=150, bbox_inches='tight')
plt.close()

print("=" * 70)
print("ENERGY MODEL FITTING (WLS)")
print("=" * 70)
print(f"\nFormula: E = α·FLOPs + β·Bytes + γ")
print(f"\nWLS Coefficients:")
print(f"  α (FLOPs):  {alpha:.6e} mJ/FLOP")
print(f"  β (Bytes):  {beta:.6e} mJ/Byte")
print(f"  γ (const):  {gamma_unscaled:.6f} mJ")
print(f"\nNon-negative WLS Coefficients:")
print(f"  α (FLOPs):  {alpha_nnls:.6e} mJ/FLOP")
print(f"  β (Bytes):  {beta_nnls:.6e} mJ/Byte")
print(f"  γ (const):  {gamma_nnls_unscaled:.6f} mJ")
print(f"\nModel Quality:")
print(f"  R²_w (weighted):      {r2_w:.4f}")
print(f"  R²_nnls_w (weighted): {r2_nnls_w:.4f}")
print(f"  R² (unweighted):      {r2_unw:.4f}")
print(f"  RMSE_w (weighted):    {rmse_w:.4f} mJ")
print(f"  RMSE (unweighted):    {rmse_unw:.4f} mJ")
print(f"\nUncertainty Analysis:")
print(f"  Model σ (RMSE_w):     {sigma_model:.4f} mJ  [weighted, WLS-consistent]")
print(f"  Pooled Meas SE:       {pooled_meas_SE:.4f} mJ")
print(f"  Total σ:              {total_uncertainty:.4f} mJ")
print(f"  Δ_95% (t={t_crit_overall:.3f}):      {delta_95_t:.4f} mJ  [PRIMARY]")
print(f"  Δ_95% (z=1.96):       {delta_95_z:.4f} mJ  [reference]")
print(f"\n  Note: RMSE_unw = {rmse_unw:.4f} mJ (unweighted, for comparison)")
print(f"        σ_resid_w = {sigma_resid_w:.4f} [unitless whitened, WLS diagnostic]")
print(f"\nUpper Bound Formula:")
print(f"  E_upper = E_point + {delta_95_t:.4f} mJ  (t-distribution)")
print("\n" + "=" * 70)
print("PREDICTIONS")
print("=" * 70)
print(f"{'Model':<20} {'Measured':<10} {'Pred(WLS)':<10} {'Upper95':<10} {'Pred(NNLS)':<11} {'Residual':<10}")
print("-" * 70)
for _, row in results_df.iterrows():
    print(f"{row['model']:<20} {row['energy_measured_mj']:>9.3f} {row['energy_predicted_mj']:>9.3f} "
          f"{row['energy_upper95_mj']:>9.3f} {row['energy_predicted_mj_nnls']:>10.3f} {row['residual_mj']:>9.3f}")
print("\n" + "=" * 70)
print("✓ WLS coefficients saved to logs/alpha_beta_gamma.json")
print("✓ NNLS coefficients saved to logs/alpha_beta_gamma_nnls.json")
print("✓ Diagnostics saved to logs/fit_diag.json")
print("✓ Fit results saved to logs/fit_results.csv")
print("✓ Diagnostic plots saved to logs/fit_diagnostics.png")
print("=" * 70)

ENERGY MODEL FITTING (WLS)

Formula: E = α·FLOPs + β·Bytes + γ

WLS Coefficients:
  α (FLOPs):  5.666402e-08 mJ/FLOP
  β (Bytes):  2.035497e-07 mJ/Byte
  γ (const):  1.734014 mJ

Non-negative WLS Coefficients:
  α (FLOPs):  5.666402e-08 mJ/FLOP
  β (Bytes):  2.035497e-07 mJ/Byte
  γ (const):  1.734014 mJ

Model Quality:
  R²_w (weighted):      0.7165
  R²_nnls_w (weighted): 0.7165
  R² (unweighted):      0.5358
  RMSE_w (weighted):    0.9315 mJ
  RMSE (unweighted):    1.6510 mJ

Uncertainty Analysis:
  Model σ (RMSE_w):     0.9315 mJ  [weighted, WLS-consistent]
  Pooled Meas SE:       0.6110 mJ
  Total σ:              1.1140 mJ
  Δ_95% (t=3.182):      3.5453 mJ  [PRIMARY]
  Δ_95% (z=1.96):       2.1835 mJ  [reference]

  Note: RMSE_unw = 1.6510 mJ (unweighted, for comparison)
        σ_resid_w = 3.3833 [unitless whitened, WLS diagnostic]

Upper Bound Formula:
  E_upper = E_point + 3.5453 mJ  (t-distribution)

PREDICTIONS
Model                Measured   Pred(WLS)  Upper95    Pred(NNLS) 

step 9

In [41]:
import os
import json
import numpy as np
import pandas as pd

os.makedirs('results', exist_ok=True)

macs_df = pd.read_csv('logs/macs_per_inf.csv')
bytes_df = pd.read_csv('logs/bytes_per_inf.csv')
calib_df = pd.read_csv('calibration/calibration_measurements.csv')

for _df in (macs_df, bytes_df, calib_df):
    if 'window_ms' in _df.columns:
        _df['window_ms'] = pd.to_numeric(_df['window_ms'], errors='coerce').astype('Int64')

with open('logs/alpha_beta_gamma.json', 'r') as f:
    coeffs = json.load(f)

with open('configs/windows.json', 'r') as f:
    window_config = json.load(f)

alpha = coeffs['alpha_flops_coefficient']
beta = coeffs['beta_bytes_coefficient']
gamma = coeffs['gamma_intercept']
delta_95 = max(0.0, float(coeffs.get('delta_95_mj', 0.0)))

if 'window_grid_ms' in window_config:
    windows_ms = window_config['window_grid_ms']
elif 'windows_ms' in window_config:
    windows_ms = window_config['windows_ms']
else:
    windows_ms = [window_config['window']['window_ms']]

windows_ms = [int(w) for w in windows_ms]
refresh_rates_hz = [float(hz) for hz in window_config['refresh_rates_hz'] if float(hz) >= 0.0]
latency_proxy_k = window_config.get('latency_proxy_ms_per_mj', 0.01)

has_window_col_macs = 'window_ms' in macs_df.columns
has_window_col_bytes = 'window_ms' in bytes_df.columns

base_window_ms = int(window_config.get('complexity_base_window_ms',
                     window_config.get('window', {}).get('window_ms', windows_ms[0])))

if has_window_col_macs:
    macs_clean = macs_df.dropna(subset=['window_ms'])
    flops_dict = {(r['model'], int(r['window_ms'])): float(r['macs_per_inf']) * 2.0
                  for _, r in macs_clean.iterrows()}
    flops_source = 'measured'
else:
    flops_base = {row['model']: float(row['macs_per_inf']) * 2.0 for _, row in macs_df.iterrows()}
    flops_dict = {(m, int(w)): flops_base[m] * (float(w) / base_window_ms)
                  for m in flops_base for w in windows_ms}
    flops_source = f'scaled_linear_from_{base_window_ms}ms'

bytes_col = 'total_bytes' if 'total_bytes' in bytes_df.columns else 'bytes_per_inf'
if has_window_col_bytes:
    bytes_clean = bytes_df.dropna(subset=['window_ms'])
    bytes_dict = {(r['model'], int(r['window_ms'])): float(r[bytes_col])
                  for _, r in bytes_clean.iterrows()}
    bytes_source = 'measured'
else:
    bytes_base = {row['model']: float(row[bytes_col]) for _, row in bytes_df.iterrows()}
    bytes_dict = {(m, int(w)): bytes_base[m] * (float(w) / base_window_ms)
                  for m in bytes_base for w in windows_ms}
    bytes_source = f'scaled_linear_from_{base_window_ms}ms'

has_window_col_calib = 'window_ms' in calib_df.columns
if has_window_col_calib:
    calib_clean = calib_df.dropna(subset=['window_ms'])
    latency_measured = {(r['model'], int(r['window_ms'])): float(r['mean_latency_ms_per_inf'])
                        for _, r in calib_clean.iterrows()}
else:
    calib_base_window = int(window_config.get('latency_base_window_ms', windows_ms[0]))
    latency_measured = {(row['model'], calib_base_window): float(row['mean_latency_ms_per_inf'])
                        for _, row in calib_df.iterrows()}

k_flops_per_ms = None
ratios = []
for (m, w), lat in latency_measured.items():
    if pd.isna(w):
        continue
    fl = flops_dict.get((m, int(w)))
    if (fl is not None) and np.isfinite(fl) and (lat is not None) and np.isfinite(lat) and (lat > 0):
        ratios.append(fl / lat)
if ratios:
    k_flops_per_ms = float(np.median(ratios))

models = sorted(set(m for m, _ in flops_dict.keys()) & set(m for m, _ in bytes_dict.keys()))

missing = [(m, w) for m in models for w in windows_ms
           if (m, w) not in flops_dict or (m, w) not in bytes_dict]
if missing:
    print(f"NOTE: skipped {len(missing)} (model,window) pairs without FLOPs/Bytes. Example: {missing[:5]}")

results = []

for model in models:
    measured_windows = {w for (m, w) in latency_measured.keys() if m == model}

    for window_ms in windows_ms:
        key = (model, window_ms)

        if key not in flops_dict or key not in bytes_dict:
            continue

        flops = flops_dict[key]
        bytes_val = bytes_dict[key]

        energy_point = max(0.0, alpha * flops + beta * bytes_val + gamma)
        energy_upper = max(energy_point, energy_point + max(0.0, float(delta_95)))

        if key in latency_measured:
            latency_ms = latency_measured[key]
            latency_source = 'measured'
            latency_base_window_ms = window_ms
        elif measured_windows:
            latency_base_window_ms = min(measured_windows, key=lambda w: abs(int(w) - int(window_ms)))
            base_latency = latency_measured[(model, latency_base_window_ms)]
            latency_ms = base_latency * (float(window_ms) / float(latency_base_window_ms))
            latency_source = f'window_linear_from_{latency_base_window_ms}ms'
        else:
            if k_flops_per_ms is not None and k_flops_per_ms > 0:
                latency_ms = flops / k_flops_per_ms
                latency_source = 'flops_proxy'
            else:
                latency_ms = energy_point * latency_proxy_k
                latency_source = f'energy_proxy_k={latency_proxy_k}'
            latency_base_window_ms = np.nan

        latency_ms = max(0.0, float(latency_ms))
        max_schedulable_hz = 1000.0 / max(latency_ms, 1e-9)
        edp = energy_point * latency_ms

        for refresh_hz in refresh_rates_hz:
            power_point_mj_per_s = energy_point * refresh_hz
            power_upper_mj_per_s = energy_upper * refresh_hz
            power_point_mw = power_point_mj_per_s
            power_upper_mw = power_upper_mj_per_s

            utilization = refresh_hz * latency_ms / 1000.0
            schedulable = utilization <= 1.0 + 1e-9

            results.append({
                'model': model,
                'window_ms': window_ms,
                'refresh_rate_hz': refresh_hz,
                'flops_per_inf': flops,
                'bytes_per_inf': bytes_val,
                'flops_source': flops_source,
                'bytes_source': bytes_source,
                'energy_point_mj_per_inf': energy_point,
                'energy_upper95_mj_per_inf': energy_upper,
                'latency_ms_per_inf': latency_ms,
                'latency_source': latency_source,
                'latency_base_window_ms': latency_base_window_ms,
                'max_schedulable_hz': max_schedulable_hz,
                'edp_mj_ms': edp,
                'power_point_mj_per_s': power_point_mj_per_s,
                'power_upper95_mj_per_s': power_upper_mj_per_s,
                'power_point_mw': power_point_mw,
                'power_upper95_mw': power_upper_mw,
                'utilization': utilization,
                'schedulable': schedulable
            })

df = pd.DataFrame(results)
df = df.sort_values(['model', 'window_ms', 'refresh_rate_hz'])
df.to_csv('results/proxy_energy_estimates.csv', index=False)

print("=" * 70)
print("PROXY ENERGY ESTIMATES")
print("=" * 70)
print(f"\nWindows: {windows_ms} ms")
print(f"Base window (for scaling): {base_window_ms} ms")
print(f"Models: {len(models)}")
print(f"Refresh rates: {refresh_rates_hz} Hz")
print(f"\nCoefficients:")
print(f"  α = {alpha:.6e} mJ/FLOP")
print(f"  β = {beta:.6e} mJ/Byte")
print(f"  γ = {gamma:.6f} mJ")
print(f"  Δ₉₅ = {delta_95:.4f} mJ (non-negative protected)")
print(f"\nComplexity sources:")
print(f"  FLOPs: {flops_source}")
print(f"  Bytes: {bytes_source}")
print(f"\nLatency extrapolation:")
print(f"  Priority: measured > window_linear > flops_proxy > energy_proxy")
if k_flops_per_ms:
    print(f"  k_flops_per_ms = {k_flops_per_ms:.2e} FLOPs/ms (median, de-duplicated)")
print(f"  Energy proxy k = {latency_proxy_k} ms/mJ")

print("\n" + "=" * 70)
print("ESTIMATES SUMMARY (per model, first window)")
print("=" * 70)
first_window = windows_ms[0]
print(f"{'Model':<20} {'E(mJ)':<10} {'Upper':<10} {'Lat(ms)':<10} {'Source':<35}")
print("-" * 70)
for model in models:
    model_data = df[(df['model'] == model) & (df['window_ms'] == first_window)]
    if len(model_data) > 0:
        row = model_data.iloc[0]
        print(f"{model:<20} {row['energy_point_mj_per_inf']:>9.3f} "
              f"{row['energy_upper95_mj_per_inf']:>9.3f} "
              f"{row['latency_ms_per_inf']:>9.3f} {row['latency_source']:<35}")

print("\n" + "=" * 70)
print("SCHEDULABILITY CHECK (Utilization = refresh_hz × latency / 1000)")
print("=" * 70)
unschedulable = df[~df['schedulable']]
if len(unschedulable) > 0:
    print(f"⚠ {len(unschedulable)} unschedulable configs (U > 1.0):")
    for _, row in unschedulable.head(10).iterrows():
        print(f"  {row['model']}, {row['window_ms']}ms, {row['refresh_rate_hz']}Hz: "
              f"U={row['utilization']:.2f}, max_hz={row['max_schedulable_hz']:.2f}")
else:
    print("✓ All configurations schedulable (with 1e-9 tolerance)")

print("\n" + "=" * 70)
print("OUTPUT COLUMNS (results/proxy_energy_estimates.csv)")
print("=" * 70)
print("  model, window_ms, refresh_rate_hz")
print("  flops_per_inf, bytes_per_inf")
print("  flops_source, bytes_source (measured / scaled_linear_from_XXms)")
print("  energy_point_mj_per_inf, energy_upper95_mj_per_inf")
print("  latency_ms_per_inf, latency_source, latency_base_window_ms")
print("  max_schedulable_hz (= 1000 / latency_ms)")
print("  edp_mj_ms (E×D product)")
print("  power_point_mj_per_s, power_upper95_mj_per_s")
print("  power_point_mw, power_upper95_mw (mW = mJ/s)")
print("  utilization, schedulable")

print("\n" + "=" * 70)
print("✓ Proxy estimates saved to results/proxy_energy_estimates.csv")
print(f"✓ Total rows: {len(df)} ({len(models)} models × {len(windows_ms)} windows × {len(refresh_rates_hz)} rates)")
print("=" * 70)

PROXY ENERGY ESTIMATES

Windows: [2560] ms
Base window (for scaling): 2560 ms
Models: 6
Refresh rates: [0.1, 0.5, 1.0, 2.0] Hz

Coefficients:
  α = 5.666402e-08 mJ/FLOP
  β = 2.035497e-07 mJ/Byte
  γ = 1.734014 mJ
  Δ₉₅ = 3.5453 mJ (non-negative protected)

Complexity sources:
  FLOPs: scaled_linear_from_2560ms
  Bytes: scaled_linear_from_2560ms

Latency extrapolation:
  Priority: measured > window_linear > flops_proxy > energy_proxy
  k_flops_per_ms = 1.91e+07 FLOPs/ms (median, de-duplicated)
  Energy proxy k = 0.01 ms/mJ

ESTIMATES SUMMARY (per model, first window)
Model                E(mJ)      Upper      Lat(ms)    Source                             
----------------------------------------------------------------------
InceptionTime            3.281     6.827     1.204 measured                           
KNN                      8.400    11.945     3.046 measured                           
MiniROCKET               3.305     6.850     0.487 measured                           
Mult

step 10

In [43]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')

os.makedirs('figures', exist_ok=True)

df = pd.read_csv('results/proxy_energy_estimates.csv')
metrics_df = pd.read_csv('logs/train_metrics.csv')

accuracy_dict = dict(zip(metrics_df['model'], metrics_df['test_accuracy']))

df['accuracy'] = df['model'].map(accuracy_dict)

fig_note = "Proxy estimates from GPU-NVML calibration; not edge-device measurements"

models = df['model'].unique()
colors = plt.cm.tab10(np.linspace(0, 1, len(models)))
model_colors = dict(zip(models, colors))

first_window = df['window_ms'].min()
first_rate = df['refresh_rate_hz'].min()

fig, ax = plt.subplots(figsize=(10, 6))
for model in models:
    model_df = df[(df['model'] == model) & (df['window_ms'] == first_window) & (df['refresh_rate_hz'] == first_rate)]
    if len(model_df) == 0:
        continue
    row = model_df.iloc[0]
    ax.scatter(row['energy_point_mj_per_inf'], row['accuracy'],
               color=model_colors[model], s=100, label=f"{model} (point)", marker='o')
    ax.scatter(row['energy_upper95_mj_per_inf'], row['accuracy'],
               color=model_colors[model], s=100, alpha=0.5, marker='x')
    ax.plot([row['energy_point_mj_per_inf'], row['energy_upper95_mj_per_inf']],
            [row['accuracy'], row['accuracy']],
            color=model_colors[model], alpha=0.3, linewidth=2)

ax.set_xlabel('Energy per Inference (mJ)', fontsize=12)
ax.set_ylabel('Test Accuracy', fontsize=12)
ax.set_title(f'Accuracy vs Energy (window={first_window}ms, rate={first_rate}Hz)', fontsize=13)
ax.legend(fontsize=9, loc='best')
ax.grid(True, alpha=0.3)
ax.text(0.02, 0.02, fig_note, transform=ax.transAxes, fontsize=8,
        verticalalignment='bottom', style='italic', color='gray')
plt.tight_layout()
plt.savefig('figures/accuracy_vs_energy.png', dpi=150, bbox_inches='tight')
plt.close()

fig, ax = plt.subplots(figsize=(10, 6))
for model in models:
    model_df = df[(df['model'] == model) & (df['window_ms'] == first_window) & (df['refresh_rate_hz'] == first_rate)]
    if len(model_df) == 0:
        continue
    row = model_df.iloc[0]
    ax.scatter(row['latency_ms_per_inf'], row['energy_point_mj_per_inf'],
               color=model_colors[model], s=100, label=model, marker='o')
    ax.errorbar(row['latency_ms_per_inf'], row['energy_point_mj_per_inf'],
                yerr=[[0], [row['energy_upper95_mj_per_inf'] - row['energy_point_mj_per_inf']]],
                fmt='none', color=model_colors[model], alpha=0.5, capsize=5)

ax.set_xlabel('Latency per Inference (ms)', fontsize=12)
ax.set_ylabel('Energy per Inference (mJ)', fontsize=12)
ax.set_title(f'Energy vs Latency (window={first_window}ms, rate={first_rate}Hz)', fontsize=13)
ax.legend(fontsize=9, loc='best')
ax.grid(True, alpha=0.3)
ax.text(0.02, 0.98, fig_note, transform=ax.transAxes, fontsize=8,
        verticalalignment='top', style='italic', color='gray')
plt.tight_layout()
plt.savefig('figures/energy_vs_latency.png', dpi=150, bbox_inches='tight')
plt.close()

windows = sorted(df['window_ms'].unique())
fig, ax = plt.subplots(figsize=(10, 6))
for model in models:
    edps = []
    for w in windows:
        model_df = df[(df['model'] == model) & (df['window_ms'] == w) & (df['refresh_rate_hz'] == first_rate)]
        if len(model_df) > 0:
            edps.append(model_df.iloc[0]['edp_mj_ms'])
        else:
            edps.append(np.nan)
    ax.plot(windows, edps, marker='o', label=model, color=model_colors[model], linewidth=2)

ax.set_xlabel('Window Duration (ms)', fontsize=12)
ax.set_ylabel('Energy-Delay Product (mJ·ms)', fontsize=12)
ax.set_title(f'EDP vs Window (rate={first_rate}Hz)', fontsize=13)
ax.legend(fontsize=9, loc='best')
ax.grid(True, alpha=0.3)
ax.text(0.02, 0.98, fig_note, transform=ax.transAxes, fontsize=8,
        verticalalignment='top', style='italic', color='gray')
plt.tight_layout()
plt.savefig('figures/edp_vs_window.png', dpi=150, bbox_inches='tight')
plt.close()

rates = sorted(df['refresh_rate_hz'].unique())
fig, ax = plt.subplots(figsize=(10, 6))
for model in models:
    powers = []
    power_uppers = []
    for r in rates:
        model_df = df[(df['model'] == model) & (df['window_ms'] == first_window) & (df['refresh_rate_hz'] == r)]
        if len(model_df) > 0:
            powers.append(model_df.iloc[0]['power_point_mj_per_s'])
            power_uppers.append(model_df.iloc[0]['power_upper95_mj_per_s'])
        else:
            powers.append(np.nan)
            power_uppers.append(np.nan)

    ax.plot(rates, powers, marker='o', label=f"{model} (point)", color=model_colors[model], linewidth=2)
    ax.fill_between(rates, powers, power_uppers, color=model_colors[model], alpha=0.2)

ax.set_xlabel('Refresh Rate (Hz)', fontsize=12)
ax.set_ylabel('Power (mJ/s = mW)', fontsize=12)
ax.set_title(f'Power vs Refresh Rate (window={first_window}ms)', fontsize=13)
ax.legend(fontsize=9, loc='best')
ax.grid(True, alpha=0.3)
ax.text(0.02, 0.98, fig_note, transform=ax.transAxes, fontsize=8,
        verticalalignment='top', style='italic', color='gray')
plt.tight_layout()
plt.savefig('figures/power_vs_refresh_rate.png', dpi=150, bbox_inches='tight')
plt.close()

print("=" * 70)
print("VISUALIZATION COMPLETE")
print("=" * 70)
print(f"\nGenerated figures:")
print(f"  1. figures/accuracy_vs_energy.png")
print(f"  2. figures/energy_vs_latency.png")
print(f"  3. figures/edp_vs_window.png")
print(f"  4. figures/power_vs_refresh_rate.png")
print(f"\nData points:")
print(f"  Models: {len(models)}")
print(f"  Windows: {windows}")
print(f"  Refresh rates: {rates}")
print("\n" + "=" * 70)
print("✓ All visualizations saved to figures/")
print("=" * 70)

VISUALIZATION COMPLETE

Generated figures:
  1. figures/accuracy_vs_energy.png
  2. figures/energy_vs_latency.png
  3. figures/edp_vs_window.png
  4. figures/power_vs_refresh_rate.png

Data points:
  Models: 6
  Windows: [np.int64(2560)]
  Refresh rates: [np.float64(0.1), np.float64(0.5), np.float64(1.0), np.float64(2.0)]

✓ All visualizations saved to figures/
